# Summarizing Data for Results Graphics in "R2: Perceived risks and benefits qualitative"

There are two central steps involved:


**Check Szenario Texts:**
- to check if topic was already mentioned for the category specific graph

**Summarizing Data:**
- for A, B graph
- for category specific graph



***
**Coding sources**

Code snippets from my (Julius Fenn) Large Language Model Workshop: https://github.com/FennStatistics/introductory-workshop-in-LLMs/


I extend the code provided and explained in the following documenations:
* Build a PDF ingestion and Question/Answering system: https://python.langchain.com/v0.2/docs/tutorials/pdf_qa/

* https://python.langchain.com/v0.2/docs/tutorials/llm_chain/ (Build a Simple LLM Application with LCEL)
* https://python.langchain.com/v0.2/docs/how_to/structured_output/ (How to return structured data from a model)
* https://python.langchain.com/v0.2/docs/how_to/llm_token_usage_tracking/ (How to track token usage for LLMs)


***
## If you facing issues running your Code:

It could be the case that chroma and langchain-core are not compatible.

In [1]:
## run in your terminal; could be necessary to downgrade package:
# pip uninstall langchain-core
# pip install langchain-core==0.3.10

## or install older version of chromadb:
# pip install --upgrade chromadb==0.5.0

## Get API, local supabase server key(s)

In [2]:
import os
import sys

# Assuming 'src' is one level down (in the current directory or a subdirectory)
path_to_src = os.path.join('src')  # Moves one level down to 'src' folder

# Add the path to sys.path
sys.path.append(path_to_src)

# Now you can import your API_key module
import API_key as key

## include self-written functions

In [3]:
import src.forChromaApproach as di_drg

# Check Szenario Texts

## Load Scenario Texts
This should be the final scenario texts in English of the two robots:

* rescue robot
* socially assistive robot

In [4]:
path_to_PDFs = os.path.join('data/scenario texts')  # Moves one level up to 'PDFs' folder

pdf_pages = di_drg.load_pdfs_by_filename(path_to_PDFs, verbose=False)

# Optional: Print the loaded pages by filename
for filename, pages in pdf_pages.items():
    print(f"\nPDF: {filename}")
    print(f"Total Pages: {len(pages)}")
    # print(pages[0])


PDF: rescue robot.pdf
Total Pages: 6

PDF: socially assistive robot.pdf
Total Pages: 6


## Data Storage: Text chunks are converted into vector embeddings and stored in a vector database (Vector DB) next to their respective text chunks.

In [5]:
pdf_chunks = di_drg.split_pdf_pages_into_chunks(pdf_pages, chunk_size=500, chunk_overlap=150, verbose=False)

# Optional: Print a summary of chunks created per PDF
for filename, chunks in pdf_chunks.items():
    print(f"\nPDF: {filename}")
    print(f"Total Chunks: {len(chunks)}")


PDF: rescue robot.pdf
Total Chunks: 15

PDF: socially assistive robot.pdf
Total Chunks: 16


In [6]:
path_to_Chroma = os.path.join('DB_Chroma')  # Moves one level up to 'PDFs' folder

sources_DB = di_drg.inspect_chrom(CHROMA_PATH=path_to_Chroma, openAI_key=key.openAI_key)
print("Number of sources in DB:", len(sources_DB))
print("\nSources:\n", sources_DB)

# Remove the "PDFs\\" prefix from all entries
cleaned_sources_DB = [pdf.replace('PDFs\\', '').replace('data/scenario texts\\', '') for pdf in sources_DB]

# Print the result
print("\nCleaned sources:\n", cleaned_sources_DB)

c:\DATEN\PHD\Article_SoftRobotIntervention\Analyses\main study - LLM\src\forChromaApproach.py:90: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=OpenAIEmbeddings(api_key=openAI_key))


Number of sources in DB: 2

Sources:
 ['data/scenario texts\\rescue robot.pdf', 'data/scenario texts\\socially assistive robot.pdf']

Cleaned sources:
 ['rescue robot.pdf', 'socially assistive robot.pdf']


In [7]:
# if you want to remove your DB:
## di_drg.remove_chrom(CHROMA_PATH=path_to_Chroma)

# pdf_chunks is a dictionary as such we can run over the keys:
for pdf in pdf_chunks.keys():
    if pdf not in cleaned_sources_DB:
        print(f"The PDF '{pdf}' is not included in the DB, as such:")
        print("create DB for", pdf)
        di_drg.save_to_chrom(chunks=pdf_chunks[pdf], CHROMA_PATH=path_to_Chroma, openAI_key=key.openAI_key)

In [8]:
sources_DB = di_drg.inspect_chrom(CHROMA_PATH=path_to_Chroma, openAI_key=key.openAI_key)
print("Number of sources in DB:", len(sources_DB))
print("\nSources:\n", sources_DB)

Number of sources in DB: 2

Sources:
 ['data/scenario texts\\rescue robot.pdf', 'data/scenario texts\\socially assistive robot.pdf']


## Data Retrieval and Generation

your prompt template (system message):

In [9]:
PROMPT_TEMPLATE = """
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"

---

Answer the question based on the above context: {question}
"""

In [10]:
# Question query 2
question = """
What are the advantages of soft robots?
"""

In [11]:
response, source_page_pairs, filtered_hits, all_hits = di_drg.retrieveGenerate(query_text=question, prompt_template=PROMPT_TEMPLATE, openAI_key=key.openAI_key, chroma_path=path_to_Chroma, 
                                                                            docsReturn=10, thresholdSimilarity=0.8)

Number of requested results 150 is greater than number of elements in index 31, updating n_results = 31


Number of possible relevant text chunks found with a threshold similarity of 0.8: 11
Query: Human: 
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "

"
    "––––––––––––  Third  Page in Experiment (Intervention) ––––––––––––   
Given the information on soft robots, we invite you to adjust your CAM regarding the 
risks and benefits of utilizing socially assistive soft robots (SASR). We are interested 
in knowing if the softness or rigidity of a robot influences your evaluation of its 
suitability for socially assistive tasks, based on the information available on soft 
robots.

---

Soft robots are designed to mimic the properties of living entities such as animals. 
Unlike rigid robots, which are typically composed of hard materials like metal or hard 
plastic, soft

c:\DATEN\PHD\Article_SoftRobotIntervention\Analyses\main study - LLM\src\forChromaApproach.py:147: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = model.predict(prompt)


In [12]:
print(response)

Soft robots have the ability to mimic the properties of living entities, making them more adaptable and lifelike compared to rigid robots. They are made of flexible materials like silicone, allowing them to bend, twist, and stretch like living organisms. Soft robots can express and perceive human emotions, communicate through expressions such as gaze and gestures, and offer a lifelike experience due to their softness and flexibility.


In [13]:
print(source_page_pairs)

[('data/scenario texts\\socially assistive robot.pdf', 5), ('data/scenario texts\\socially assistive robot.pdf', 3), ('data/scenario texts\\socially assistive robot.pdf', 3), ('data/scenario texts\\rescue robot.pdf', 4), ('data/scenario texts\\rescue robot.pdf', 4), ('data/scenario texts\\rescue robot.pdf', 4), ('data/scenario texts\\rescue robot.pdf', 5), ('data/scenario texts\\rescue robot.pdf', 1), ('data/scenario texts\\rescue robot.pdf', 3), ('data/scenario texts\\socially assistive robot.pdf', 3)]


In [14]:
print(len(all_hits))
print(all_hits[0])

11
(Document(metadata={'page': 4, 'source': 'data/scenario texts\\rescue robot.pdf'}, page_content='––––––––––––  Second  Page in Experiment (Intervention) ––––––––––––   \nSoft Robots for Search and Rescue Missions  \nBenefits of soft robots for search and rescue missions might be:  \n●  Access to areas unreachable or too dangerous for human rescuers  \n●  Delivery  of essential supplies (water, food, medicine) until victims are safely  \nextracted  \n●  Reduced risk of injury to victims due to their flexibility and adaptability'), 0.8405289303258905)


# Summarizing Data
- for A, B graph
- for category specific graph


## Load .xlsx files (lists of words)

* rescue robot_multipleSheets
* socially assistive robot_multipleSheets
* rescue robot_socially assistive robot_multipleSheets

> there are seperate columns for new (added concepts), deleted (deleted concepts), constant (not changed concepts)


In [15]:
## set working environment
#> Get the current working directory
print(os.getcwd())
directory = os.getcwd()

c:\DATEN\PHD\Article_SoftRobotIntervention\Analyses\main study - LLM


In [16]:
import pandas as pd

## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "rescue robot_social assistance robot_multipleSheets_new" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names combined:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_Combined = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}



## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "rescue robot_multipleSheets_new" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names RR:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_RR = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}

## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "social assistance robot_multipleSheets_new" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names SAR:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_SAR = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}

Sheet names combined: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']
Sheet names RR: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']
Sheet names SAR: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']


## Additional dictonaries to provide LLM context

In [17]:
abbreviations_dict = {
    'RCPP': 'perceived positive usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)',
    'LC': 'perceived low costs',
    'T': 'perceived trust',
    'SIP': 'perceived positive social impact',
    'HRIP': 'perceived positive Human-Robot-Interaction',
    'AN': 'perceived negative anthropomorphism',
    'SIN': 'perceived positive social impact',
    'R': 'perceived risks',
    'HC': 'perceived high costs',
    'RCN': 'neutral rest category (rest category refers to a classification of arguments that do not fit into any of the predefined categories)',
    'SA': 'perceived safety',
    'TP': 'perceived technological possibilities',
    'TL': 'perceived technological limitations',
    'RCPN': 'perceived negative usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)',
    'HRIN': 'perceived negative Human-Robot-Interaction',
    'MT': 'perceived mistrust',
    'RCA': 'ambivalent rest category (rest category refers to a classification of arguments that do not fit into any of the predefined categories)',
    'AP': 'perceived positive anthropomorphism'
}

print(abbreviations_dict)
print(abbreviations_dict.keys())
print(abbreviations_dict['AN'])

{'RCPP': 'perceived positive usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)', 'LC': 'perceived low costs', 'T': 'perceived trust', 'SIP': 'perceived positive social impact', 'HRIP': 'perceived positive Human-Robot-Interaction', 'AN': 'perceived negative anthropomorphism', 'SIN': 'perceived positive social impact', 'R': 'perceived risks', 'HC': 'perceived high costs', 'RCN': 'neutral rest category (rest category refers to a classification of arguments that do not fit into any of the predefined categories)', 'SA': 'perceived safety', 'TP': 'perceived technological possibilities', 'TL': 'perceived technological limitations', 'RCPN': 'perceived negative usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)', 'HRIN': 'perceived negative Human-Robot-Interaction', 'MT': 'perceived mistrust', 'RCA': 'ambivalent rest category (rest category refers to a c

keep only subset of list (which have meaningfull differences):

In [18]:
# Keys to keep
keys_to_keep = ['TP', 'TL', 'SA', 'R', 'HRIP', 'HRIN', 'AP', 'AN']

# Filter the dictionary
abbreviations_dict = {key: abbreviations_dict[key] for key in keys_to_keep}

# Display the filtered dictionary
print(abbreviations_dict)

{'TP': 'perceived technological possibilities', 'TL': 'perceived technological limitations', 'SA': 'perceived safety', 'R': 'perceived risks', 'HRIP': 'perceived positive Human-Robot-Interaction', 'HRIN': 'perceived negative Human-Robot-Interaction', 'AP': 'perceived positive anthropomorphism', 'AN': 'perceived negative anthropomorphism'}


Use dictionaries to map each word to its comment:

In [19]:
import numpy as np

def create_multivalue_dict(df, key_col, value_col):
    """
    Create a dictionary from a DataFrame where each key maps to a list of values.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    key_col (str): The column name to be used as keys.
    value_col (str): The column name to be used as values.
    
    Returns:
    dict: A dictionary where each key maps to a list of values.
    """
    # Remove rows with NaN in the key columns
    df = df.dropna(subset=[key_col])

    # Create a dictionary to map items to their comments, allowing for multiple comments per key
    multivalue_dict = {}
    for key, value in zip(df[key_col], df[value_col]):
        if key in multivalue_dict:
            multivalue_dict[key].append(value)
        else:
            multivalue_dict[key] = [value]

    return multivalue_dict

# Example usage
data = {
    'constant': ['a', 'b', 'a', 'c', 'b', np.nan],
    'constant_comments': ['comment1', 'comment2', 'comment3', 'comment4', 'comment5', 'comment6']
}
df = pd.DataFrame(data)

df_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
print(df_mapping)


{'a': ['comment1', 'comment3'], 'b': ['comment2', 'comment5'], 'c': ['comment4']}


test create_multivalue_dict() function:

In [20]:
sheet_name = "SA"

In [21]:
# Remove rows with NaN in the key columns as they cannot be used as dictionary keys
#> not sensitive to multiple identical keys: dict(zip(df['constant'], df['constant_comments']))
df = all_sheets_Combined[sheet_name]

constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
print("mapping constant x comments:", constant_comments_mapping)
print(len(constant_comments_mapping))

new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
print("mapping new x comments:", new_comments_mapping)
print(len(new_comments_mapping))

deleted_comments_mapping = create_multivalue_dict(df, 'deleted', 'deleted_comments')
print("mapping deleted x comments:", deleted_comments_mapping)
print(len(deleted_comments_mapping))

mapping constant x comments: {'quick hygienic help': [nan], 'efficient': ['Work on statistics can be made more efficient through probabilities.', nan, nan, nan], 'safety': ['Prevents people from having to work under difficult or dangerous conditions to increase safety.', 'Enables deployment in difficult situations where human rescuers would put themselves in danger', nan, nan, nan, 'through reliability', 'No people are being put in danger'], 'remote control': ['Can be operated by experts via remote control or autonomous systems to perform operations from a safe distance.', nan, nan, nan, nan, 'Minimizes risks associated with autonomy'], 'speed and efficiency': ['Robots can often act faster than humans and increase the speed of emergency interventions', 'Improvement of speed and efficiency of rescue operations', nan], 'safe': ['People do not have to put themselves in dangerous situations'], 'reliable': ['Can be precisely controlled', nan, nan, nan, nan], 'health monitoring': [nan, 'Peop

In [22]:
def combine_dicts(dict1, dict2):
    """
    Combine two dictionaries where each key maps to a list of values.
    
    Parameters:
    dict1 (dict): The first dictionary.
    dict2 (dict): The second dictionary.
    
    Returns:
    dict: A combined dictionary where each key maps to a concatenated list of values.
    """
    combined_dict = dict1.copy()
    for key, values in dict2.items():
        if key in combined_dict:
            combined_dict[key].extend(values)
        else:
            combined_dict[key] = values
    return combined_dict

test combine_dicts() function:

In [23]:
constant_new_comments_mapping = combine_dicts(constant_comments_mapping, new_comments_mapping)
print("mapping constant, new x comments:", constant_new_comments_mapping)
print(len(constant_new_comments_mapping))


mapping constant, new x comments: {'quick hygienic help': [nan], 'efficient': ['Work on statistics can be made more efficient through probabilities.', nan, nan, nan], 'safety': ['Prevents people from having to work under difficult or dangerous conditions to increase safety.', 'Enables deployment in difficult situations where human rescuers would put themselves in danger', nan, nan, nan, 'through reliability', 'No people are being put in danger', 'Soft material reduces risk of injury'], 'remote control': ['Can be operated by experts via remote control or autonomous systems to perform operations from a safe distance.', nan, nan, nan, nan, 'Minimizes risks associated with autonomy'], 'speed and efficiency': ['Robots can often act faster than humans and increase the speed of emergency interventions', 'Improvement of speed and efficiency of rescue operations', nan], 'safe': ['People do not have to put themselves in dangerous situations'], 'reliable': ['Can be precisely controlled', nan, nan

## Data for A, B graph (G2)

Prompt to get list of arguments and explenations:

In [24]:
from langchain_core.prompts import ChatPromptTemplate


system_template = """
You are a researcher summarizing two word lists that represent people's assessments of rigid and soft robots, whereby laypersons were informed about the risks and benefits of {robots} through scenario texts.

Participants shared their views on traditional rigid robots in a "rigid" list and on flexible, electronic-free soft robots in a "soft" list after learning about their respective risks and benefits. 
The overall theme is {topicCategory}.

Both "rigid" and "soft" lists are dictionaries with argument keys and comment values. If [nan] appears, it means no comment was provided; repeated entries or [nan] values indicate that the argument was emphasized multiple times.

Your task:
Summarize the main points for each category into a JSON object. Fill the two arrays within the JSON object, "rigid_arguments" for the "rigid" list and "soft_arguments" for the "soft" list. 
Summarize the main points for each category in a JSON object. Populate the two arrays, "rigid_arguments" and "soft_arguments," with up to five key arguments each. 
Include a brief, interrelated explanation (up to two sentences) for each argument, derived from the provided lists.


Output Format:

{{
  "assessments": {{
    "rigid_arguments": [
      {{
        "argument": "argument1",
        "explanation": "explanation of argument1"
      }},
      {{
        "argument": "argument2",
        "explanation": "explanation of argument2"
      }},
      ...
    ],
    "soft_arguments": [
      {{
        "argument": "argument1",
        "explanation": "explanation of argument1"
      }},
      {{
        "argument": "argument2",
        "explanation": "explanation of argument2"
      }},
      ...
    ]
  }}
}}

Please respond with the entire JSON structure as specified, providing up to five arguments for each list, and without any additional commentary or context.
"""


user_template = """List "rigid": 
{rigid}

List "soft": 
{soft}"""

# rescue robots and socially assistive robots
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

template_out = prompt_template.invoke({"robots": "rescue robots and socially assistive robots", "topicCategory": abbreviations_dict[sheet_name], "rigid": constant_comments_mapping, "soft": new_comments_mapping})
print(template_out)

print("template_out:", template_out)
print("template_out.to_messages():", template_out.to_messages())

messages=[SystemMessage(content='\nYou are a researcher summarizing two word lists that represent people\'s assessments of rigid and soft robots, whereby laypersons were informed about the risks and benefits of rescue robots and socially assistive robots through scenario texts.\n\nParticipants shared their views on traditional rigid robots in a "rigid" list and on flexible, electronic-free soft robots in a "soft" list after learning about their respective risks and benefits. \nThe overall theme is perceived safety.\n\nBoth "rigid" and "soft" lists are dictionaries with argument keys and comment values. If [nan] appears, it means no comment was provided; repeated entries or [nan] values indicate that the argument was emphasized multiple times.\n\nYour task:\nSummarize the main points for each category into a JSON object. Fill the two arrays within the JSON object, "rigid_arguments" for the "rigid" list and "soft_arguments" for the "soft" list. \nSummarize the main points for each catego

### Single Run

Function to call LLM:

In [25]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

def huggingface_API_call(
    prompt,
    
    robots,
    topicCategory,
    dictonaryRigid,
    dictonarySoft,
    
    api_key=key.hugging_api_key,

    model_name="meta-llama/Meta-Llama-3-70B-Instruct",
    json_schema=None,
    max_tokens=1000,
    temperature=0.2,
    verbose=True
):

    # Initialize the ChatOpenAI model for Hugging Face
    model = ChatOpenAI(
        model=model_name,
        openai_api_key=api_key,
        openai_api_base="https://api-inference.huggingface.co/v1/",
        max_tokens=max_tokens,
        temperature=temperature
    )

    # Check if structured output is required and configure it
    if json_schema:
        structured_llm = model.with_structured_output(json_schema, include_raw=True)
        chain = prompt | structured_llm
    else:
        chain = prompt | model

    # Execute the model and output response details
    with get_openai_callback() as cb:
        response = chain.invoke(
            {"robots": robots, "topicCategory": topicCategory, "rigid": dictonaryRigid, "soft": dictonarySoft}
        )
        
        if cb.total_tokens > max_tokens:
            print("Warning: The response may be incomplete due to exceeding the maximum token limit.")
        
        if verbose:
            print(cb)
            print(f"Total Tokens: {cb.total_tokens}")
            print(f"Prompt Tokens: {cb.prompt_tokens}")
            print(f"Completion Tokens: {cb.completion_tokens}")
            print(f"Total Cost (USD): ${cb.total_cost}")

    return response

In [26]:
response = huggingface_API_call(
    prompt=prompt_template,
    
    robots="rescue robots and socially assistive robots",
    topicCategory=abbreviations_dict[sheet_name],
    dictonaryRigid=constant_comments_mapping,
    dictonarySoft=new_comments_mapping,
    
    api_key=key.hugging_api_key,
    model_name="meta-llama/Meta-Llama-3-70B-Instruct",

    json_schema=None,
    max_tokens=4000,
    temperature=0.2,
    verbose=True
)

Tokens Used: 4413
	Prompt Tokens: 3949
	Completion Tokens: 464
Successful Requests: 1
Total Cost (USD): $0.0
Total Tokens: 4413
Prompt Tokens: 3949
Completion Tokens: 464
Total Cost (USD): $0.0


result - structured JSON output:

In [27]:
response

AIMessage(content='{\n  "assessments": {\n    "rigid_arguments": [\n      {\n        "argument": "safety",\n        "explanation": "Rigid robots can prevent people from having to work under difficult or dangerous conditions, increasing safety and reducing the risk of injury or death."\n      },\n      {\n        "argument": "efficiency",\n        "explanation": "Rigid robots can work faster and more efficiently than humans, allowing for quicker emergency interventions and improved rescue operations."\n      },\n      {\n        "argument": "reliability",\n        "explanation": "Rigid robots can be precisely controlled and are always ready to perform their tasks, making them a reliable option for rescue operations."\n      },\n      {\n        "argument": "strength",\n        "explanation": "Rigid robots can perform tasks that require great strength, such as lifting debris, and can operate in environments that would be toxic to humans."\n      },\n      {\n        "argument": "accessib

In [28]:
response.content

'{\n  "assessments": {\n    "rigid_arguments": [\n      {\n        "argument": "safety",\n        "explanation": "Rigid robots can prevent people from having to work under difficult or dangerous conditions, increasing safety and reducing the risk of injury or death."\n      },\n      {\n        "argument": "efficiency",\n        "explanation": "Rigid robots can work faster and more efficiently than humans, allowing for quicker emergency interventions and improved rescue operations."\n      },\n      {\n        "argument": "reliability",\n        "explanation": "Rigid robots can be precisely controlled and are always ready to perform their tasks, making them a reliable option for rescue operations."\n      },\n      {\n        "argument": "strength",\n        "explanation": "Rigid robots can perform tasks that require great strength, such as lifting debris, and can operate in environments that would be toxic to humans."\n      },\n      {\n        "argument": "accessibility",\n        "

In [29]:
import json
import re

try:
    # Attempt to parse the response directly as JSON
    data = json.loads(response.content)
    # print("Valid JSON object:", json.dumps(data, indent=2))
except json.JSONDecodeError:
    # If not valid JSON, handle extraction using regex to match JSON block between triple backticks (```)
    json_match = re.search(r'```(.*?)```', response.content, re.DOTALL)
    # If JSON block is found, parse it
    if json_match:
        json_text = json_match.group(1).strip()  # Extract JSON text and strip whitespace
        try:
            data = json.loads(json_text)   # Parse JSON
            print("Valid JSON object after regex:", json.dumps(data, indent=2))
        except json.JSONDecodeError as e:
            print("Failed to parse JSON:", e)
    else:
        print("No JSON object found.")

In [30]:
# Extract rigid and soft arguments and format them into a DataFrame
arguments = []
for category, items in data['assessments'].items():
    for item in items:
        arguments.append({
            'type': category.split('_')[0],  # Extracts 'rigid' or 'soft' from 'rigid_arguments'/'soft_arguments'
            'argument': item['argument'],
            'explanation': item['explanation']
        })

# Create DataFrame
df = pd.DataFrame(arguments)

df["category"] = sheet_name

# Display the DataFrame
print(df)

    type              argument  \
0  rigid                safety   
1  rigid            efficiency   
2  rigid           reliability   
3  rigid              strength   
4  rigid         accessibility   
5   soft  lower risk of injury   
6   soft           flexibility   
7   soft           reliability   
8   soft          adaptability   
9   soft       care of victims   

                                         explanation category  
0  Rigid robots can prevent people from having to...       SA  
1  Rigid robots can work faster and more efficien...       SA  
2  Rigid robots can be precisely controlled and a...       SA  
3  Rigid robots can perform tasks that require gr...       SA  
4  Rigid robots can access areas that are difficu...       SA  
5  Soft robots pose a lower risk of injury to vic...       SA  
6  Soft robots are flexible and can access narrow...       SA  
7  Soft robots are reliable and can perform tasks...       SA  
8  Soft robots can adapt to different situations 

### Multiple Runs

Higher order function to call LLM for all categories:

In [31]:
import json
import pandas as pd
import re

def process_robot_data(type_robot):
    # Define naming based on type_robot
    if type_robot == "RR":
        all_sheets = all_sheets_RR
        naming_robots = "rescue robots"
    elif type_robot == "SAR":
        all_sheets = all_sheets_SAR
        naming_robots = "socially assistive robots"
    elif type_robot == "Combined":
        all_sheets = all_sheets_Combined
        naming_robots = "rescue robots and socially assistive robots"
    else:
        raise ValueError("Invalid type_robot specified.")

    # Initialize an empty DataFrame for concatenation
    final_df = pd.DataFrame()

    for index, category in enumerate(abbreviations_dict.keys()):
        print(f"index: {index}, category: {category}")

        # Load the specific DataFrame for the current category
        df = all_sheets[category]

        # Generate constant and new comments mappings
        constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
        new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')

        # Call API with specified parameters
        response = huggingface_API_call(
            prompt=prompt_template,
            robots=naming_robots,
            topicCategory=abbreviations_dict[category],
            dictonaryRigid=constant_comments_mapping,
            dictonarySoft=new_comments_mapping,
            api_key=key.hugging_api_key,
            model_name="meta-llama/Meta-Llama-3-70B-Instruct",
            json_schema=None,
            max_tokens=4200,
            temperature=0.0,
            verbose=False
        )
        
        # print("response.content:\n", response.content)
        #  data = json.loads(response.content)
        # Regular expression to match JSON block between triple backticks (```)
        try:
            # Attempt to parse the response directly as JSON
            data = json.loads(response.content)
            # print("Valid JSON object:", json.dumps(data, indent=2))
        except json.JSONDecodeError:
            # If not valid JSON, handle extraction using regex to match JSON block between triple backticks (```)
            json_match = re.search(r'```(.*?)```', response.content, re.DOTALL)
            # If JSON block is found, parse it
            if json_match:
                json_text = json_match.group(1).strip()  # Extract JSON text and strip whitespace
                try:
                    data = json.loads(json_text)   # Parse JSON
                    # print("Valid JSON object after regex:", json.dumps(data, indent=2))
                except json.JSONDecodeError as e:
                    print("Failed to parse JSON:", e)
                    # break # !!!
            else:
                print("No JSON object found.")
                # break # !!!

        # Extract arguments and format them into a temporary DataFrame
        arguments = []
        for arg_category, items in data['assessments'].items():
            for item in items:
                arguments.append({
                    'type': arg_category.split('_')[0],
                    'argument': item['argument'],
                    'explanation': item['explanation']
                })

        df_tmp = pd.DataFrame(arguments)
        df_tmp["category"] = category
        
        print(f"length of df_tmp: {len(df_tmp)}")

        # Concatenate the current DataFrame to the final DataFrame
        final_df = pd.concat([final_df, df_tmp], ignore_index=True)

    return final_df

logic if process_robot_data() should be run:

In [32]:
run_process_robot_data = False # True

for rescue robots:

In [33]:
# Path to your Excel file
file_path = directory + "/output/G2/" + "rescue robots" + ".xlsx"

if run_process_robot_data:
    df_RR = process_robot_data(type_robot="RR")
    # save the dataframe to an Excel file
    df_RR.to_excel(file_path, index=False)
else:
    df_RR = pd.read_excel(file_path)

for socially assistive robots:

In [34]:
# Path to your Excel file
file_path = directory + "/output/G2/" + "socially assistive robots" + ".xlsx"
    
if run_process_robot_data:
    df_SAR = process_robot_data(type_robot="SAR")
    # save the dataframe to an Excel file
    df_SAR.to_excel(file_path, index=False)
else:
    df_SAR = pd.read_excel(file_path)

for rescue robots and socially assistive robots:

In [35]:
# Path to your Excel file
file_path = directory + "/output/G2/" + "rescue robots AND socially assistive robots" + ".xlsx"

if run_process_robot_data:
    df_Combined = process_robot_data(type_robot="Combined")
    # save the dataframe to an Excel file
    df_Combined.to_excel(file_path, index=False)
else:
    df_Combined = pd.read_excel(file_path)

### Summarize Generated Data for A, B graph (G2)

In [36]:
tmp_rigid = df_RR[(df_RR["category"] == "TP") & (df_RR["type"] == "rigid")]
tmp_soft = df_RR[(df_RR["category"] == "TP") & (df_RR["type"] == "soft")]

tmp_string_rigid = 'Arguments for "rigid" robots:'
for index, row in tmp_rigid.iterrows():
    tmp_string_rigid += " \n " + row["argument"]
    tmp_string_rigid += ": " + row["explanation"]
    
tmp_string_soft = 'Arguments for "soft" robots:'
for index, row in tmp_soft.iterrows():
    tmp_string_soft += " \n " + row["argument"]
    tmp_string_soft += ": " + row["explanation"]

In [37]:
tmp_string_rigid

'Arguments for "rigid" robots: \n new places: Rigid robots can search in places where people cannot reach, such as underwater caves or narrow openings. \n resilience: Rigid robots can withstand adverse conditions, making them more effective in disaster areas. \n special abilities: Rigid robots can perform special tasks like flying, hacking doors, or sending images to the control center with cameras. \n environment-independent: Rigid robots can operate in various environments, including air, water, and ground, and can withstand toxic or narrow environments. \n more power: Rigid robots can have more strength than humans, allowing them to perform tasks that require heavy lifting or drilling.'

In [38]:
tmp_string_soft

'Arguments for "soft" robots: \n deliver supplies: Soft robots can deliver essential goods like food, water, and medicine to victims in hard-to-reach areas. \n accessible: Soft robots can reach inaccessible places due to their small size and high flexibility, allowing them to supply victims with vital resources. \n care for victims: Soft robots can provide care for victims during the rescue operation, such as delivering food and medicine. \n adaptability: Soft robots can adapt to complex problems and changing situations, making them effective in disaster areas. \n temporary supply: Soft robots can provide temporary supply of vital resources to victims until human rescuers arrive.'

Prompt to get summary (focus on overlapping, diverging arguments) for A, B graph regarding single categories:

In [39]:
from langchain_core.prompts import ChatPromptTemplate


system_template = """
You are a researcher summarizing central arguments and their explenations of people's assessments of rigid and soft robots, 
whereby laypersons were informed about the risks and benefits of {robots} through scenario texts.

Participants shared their central arguments and explenations on traditional rigid robots in arguments for "rigid" robots 
and on flexible, electronic-free soft robots in arguments for "soft" robots.

The overall theme of these arguments is {topicCategory}.

Your task:

Write a paragraph highlighting the commonalities of the arguments for rigid and soft robots followed by a brief discussion of the main differences, 
focusing stronger on the arguments for soft robots. 

The paragraph should be limited to four sentences. Provide only the paragraph without any additional commentary or context.
"""


user_template = """arguments for "rigid" robots: 
{rigid}

arguments for "soft" robots: 
{soft}"""

# rescue robots and socially assistive robots
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

template_out = prompt_template.invoke({"robots": "rescue robots", "topicCategory": abbreviations_dict["TP"], "rigid": tmp_string_rigid, "soft": tmp_string_soft})
print(template_out)

print("template_out:", template_out)
print("template_out.to_messages():", template_out.to_messages())

messages=[SystemMessage(content='\nYou are a researcher summarizing central arguments and their explenations of people\'s assessments of rigid and soft robots, \nwhereby laypersons were informed about the risks and benefits of rescue robots through scenario texts.\n\nParticipants shared their central arguments and explenations on traditional rigid robots in arguments for "rigid" robots \nand on flexible, electronic-free soft robots in arguments for "soft" robots.\n\nThe overall theme of these arguments is perceived technological possibilities.\n\nYour task:\n\nWrite a paragraph highlighting the commonalities of the arguments for rigid and soft robots followed by a brief discussion of the main differences, \nfocusing stronger on the arguments for soft robots. \n\nThe paragraph should be limited to four sentences. Provide only the paragraph without any additional commentary or context.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='arguments for "rigid" robots: \n

#### Single Run

Function to call LLM:

In [40]:
response = huggingface_API_call(
    prompt=prompt_template,
    
    robots="rescue robots",
    topicCategory=abbreviations_dict["TP"],
    dictonaryRigid=tmp_string_rigid,
    dictonarySoft=tmp_string_soft,
    
    api_key=key.hugging_api_key,
    model_name="meta-llama/Meta-Llama-3-70B-Instruct",

    json_schema=None,
    max_tokens=4000,
    temperature=0.2,
    verbose=True
)

Tokens Used: 569
	Prompt Tokens: 444
	Completion Tokens: 125
Successful Requests: 1
Total Cost (USD): $0.0
Total Tokens: 569
Prompt Tokens: 444
Completion Tokens: 125
Total Cost (USD): $0.0


In [41]:
response

AIMessage(content="The arguments for both rigid and soft robots share a common thread of emphasizing their potential to access and operate in challenging environments, thereby enhancing rescue efforts. Both types of robots are seen as capable of reaching areas inaccessible to humans, whether due to their size, flexibility, or ability to withstand adverse conditions. However, the arguments for soft robots diverge in their focus on the provision of care and supply of essential resources to victims, highlighting their potential for more nuanced and supportive roles in disaster scenarios. Overall, the emphasis on soft robots' adaptability and ability to provide temporary supply of vital resources underscores their unique value in rescue operations.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 444, 'total_tokens': 569, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Meta-Llam

In [42]:
response.content

"The arguments for both rigid and soft robots share a common thread of emphasizing their potential to access and operate in challenging environments, thereby enhancing rescue efforts. Both types of robots are seen as capable of reaching areas inaccessible to humans, whether due to their size, flexibility, or ability to withstand adverse conditions. However, the arguments for soft robots diverge in their focus on the provision of care and supply of essential resources to victims, highlighting their potential for more nuanced and supportive roles in disaster scenarios. Overall, the emphasis on soft robots' adaptability and ability to provide temporary supply of vital resources underscores their unique value in rescue operations."

### Multiple Runs

Higher order function to call LLM for all categories:

In [43]:
import pandas as pd

def summarize_processed_robot_data(type_robot):
    # Define naming based on type_robot
    if type_robot == "RR":
        tmp_df = df_RR
        naming_robots = "rescue robots"
    elif type_robot == "SAR":
        tmp_df = df_SAR
        naming_robots = "socially assistive robots"
    elif type_robot == "Combined":
        tmp_df = df_Combined
        naming_robots = "rescue robots and socially assistive robots"
    else:
        raise ValueError("Invalid type_robot specified.")


    # Initialize an empty DataFrame for concatenation
    data_array = []
    
    for index, category in enumerate(abbreviations_dict.keys()):
        print(f"index: {index}, category: {category}")

        # Load the specific DataFrame for the current category
        tmp_rigid = tmp_df[(tmp_df["category"] == category) & (tmp_df["type"] == "rigid")]
        tmp_soft = tmp_df[(tmp_df["category"] == category) & (tmp_df["type"] == "soft")]
        
        tmp_string_rigid = 'Arguments for "rigid" robots:'
        for index, row in tmp_rigid.iterrows():
            tmp_string_rigid += " \n " + row["argument"]
            tmp_string_rigid += ": " + row["explanation"]

        tmp_string_soft = 'Arguments for "soft" robots:'
        for index, row in tmp_soft.iterrows():
            tmp_string_soft += " \n " + row["argument"]
            tmp_string_soft += ": " + row["explanation"]
        
        
        # Call API with specified parameters
        response = huggingface_API_call(
            prompt=prompt_template,
            
            robots=naming_robots,
            topicCategory=abbreviations_dict[category],
            dictonaryRigid=tmp_string_rigid,
            dictonarySoft=tmp_string_soft,
            
            api_key=key.hugging_api_key,
            model_name="meta-llama/Meta-Llama-3-70B-Instruct",

            json_schema=None,
            max_tokens=4200,
            temperature=0.0,
            verbose=False
        )
        
        data_array.append({'category': category, 'summary':response.content})

    final_df = pd.DataFrame(data_array)
    return final_df

In [44]:
run_summary_processed_robot_data = False # True

for rescue robots - summary:

In [45]:
# Path to your Excel file
file_path = directory + "/output/G2/" + "rescue robots summary" + ".xlsx"

if run_summary_processed_robot_data:
    df_RR_summary = summarize_processed_robot_data(type_robot="RR")
    # save the dataframe to an Excel file
    df_RR_summary.to_excel(file_path, index=False)
else:
    df_RR_summary = pd.read_excel(file_path)

for socially assistive robots - summary:

In [46]:
# Path to your Excel file
file_path = directory + "/output/G2/" + "socially assistive robots summary" + ".xlsx"
    
if run_summary_processed_robot_data:
    df_SAR_summary = summarize_processed_robot_data(type_robot="SAR")
    # save the dataframe to an Excel file
    df_SAR_summary.to_excel(file_path, index=False)
else:
    df_SAR_summary = pd.read_excel(file_path)

## Data for Category Specific Graph (G3)

Prompt to get coding guidelines (bottom up):

code copied from:

In [47]:
"""

Generic LLM helpers:

- querying with a list of multiple prompts
- caching LLM responses to save API costs
- tokenization
- handling different APIs

"""


import random
import pandas as pd
import os
import openai
from openai import AzureOpenAI
import httpx
import numpy as np
import pickle
import hashlib
import json
import scipy
import shutil
import tiktoken
import re
import asyncio
import time
import json
from itertools import chain
from sklearn.neighbors import NearestNeighbors


# globals: OpenAI client instances
client = None
embed_client = None
client_async = None
API_type = None
__all__=["API_type","client","embed_client","client_async"]

'''
Set up rewriting the base path with Aalto mappings
For all endpoints see https://www.aalto.fi/en/services/azure-openai#6-available-api-s
'''
# prior to making any Aalto API requests, we will update this with the desired model's OpenAI name
current_openai_model = None
# mapping from OpenAI model names to Aalto API URLs
openai2aalto = {
    "gpt-3.5-turbo": "/v1/chat",
    "gpt-4-turbo": "/v1/openai/gpt4-turbo/chat/completions",
    "gpt-4o": "/v1/openai/gpt4o/chat/completions",
    "text-embedding-3-large": "/v1/openai/text-embedding-3-large/embeddings",
    "text-embedding-ada-002": "/v1/openai/ada-002/embeddings"
}


def update_base_url_for_aalto(request: httpx.Request) -> None:
    '''
    A callback that the Aalto OpenAI clients will use to append the base API url with model URL
    '''
    if request.url.path == "/chat/completions":
        if current_openai_model not in openai2aalto:
            raise Exception(f"Model {current_openai_model} not available via the Aalto API")
        request.url = request.url.copy_with(path=openai2aalto[current_openai_model])


def init(API):
    '''
    This must be called before calling QueryLLM or other methods that make GPT API calls.

    :param API: Either "OpenAI" or "Aalto"
    '''
    global client
    global embed_client
    global client_async
    global API_type
    API_type = API

    if API == "OpenAI":
        assert (
                "OPENAI_API_KEY" in os.environ and os.environ.get("OPENAI_API_KEY") != ""
        ), "you must set the `OPENAI_API_KEY` environment variable."
        client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        embed_client = client
        client_async = openai.AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    elif API == "Aalto":
        # create chat client
        assert (
                "AALTO_OPENAI_API_KEY" in os.environ and os.environ.get("AALTO_OPENAI_API_KEY") != ""
        ), "you must set the `AALTO_OPENAI_API_KEY` environment variable."

        client = openai.OpenAI(
            base_url="https://aalto-openai-apigw.azure-api.net",
            api_key=False,  # API key not used, and rather set below
            default_headers={
                "Ocp-Apim-Subscription-Key": os.environ.get("AALTO_OPENAI_API_KEY"),
            },
            http_client=httpx.Client(
                event_hooks={"request": [update_base_url_for_aalto]}
            ),
        )

        # create embedding client
        auth_headers = {
            'Ocp-Apim-Subscription-Key': os.environ.get("AALTO_OPENAI_API_KEY")
        }
        embed_client = AzureOpenAI(
            api_key="not_in_use",  # This attribute is required but it is not in use
            api_version="2024-06-01",
            azure_endpoint="https://aalto-openai-apigw.azure-api.net/v1/",
            default_headers=auth_headers
        )
        '''
        assert (
                "OPENAI_API_KEY" in os.environ and os.environ.get("OPENAI_API_KEY") != ""
        ), "you must set the `OPENAI_API_KEY` environment variable."


        #Aalto embedding models give an error: {'statusCode': 404, 'message': 'Resource not found'}
        #For now, we use OpenAI embeddings instead, which should be GDPR-safe because we only embed codes instead of the raw data
        assert (
                "OPENAI_API_KEY" in os.environ and os.environ.get("OPENAI_API_KEY") != ""
        ), "you must set the `OPENAI_API_KEY` environment variable."
        embed_client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


        #Asynchronous processing disabled for now, as it gives weird errors with Aalto models
        client_async=None

        client_async = openai.AsyncAzureOpenAI(
            base_url="https://aalto-openai-apigw.azure-api.net",
            api_key=os.environ.get("AALTO_OPENAI_API_KEY"), #False, # API key not used, and rather set below
            api_version="2023-05-15",
            default_headers = {
                "Ocp-Apim-Subscription-Key": os.environ.get("AALTO_OPENAI_API_KEY"),
            },
            http_client=httpx.AsyncClient(
                event_hooks={ "request": [update_base_url_for_aalto] }
            ),
        )
        '''
    else:
        raise Exception(f"Invalid LLM API: {API}")


# progress bar helper
def print_progress_bar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█', printEnd="\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()


cache_dir = "./_LLMCode_cache" #os.path.join(os.path.dirname(os.path.realpath(__file__)), "../_LLM_cache")


def cache_keys_equal(key1, key2):
    if (type(key1) is np.ndarray) and (type(key2) is np.ndarray):
        return np.array_equal(key1, key2)
    return key1 == key2


def cache_hash(key):
    return hashlib.md5(key).hexdigest()


def load_cached(key):
    cached_name = cache_dir + "/" + cache_hash(key)
    if os.path.exists(cached_name):
        cached = pickle.load(open(cached_name, "rb"))
        if cache_keys_equal(cached["key"], key):
            # cache_copy_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)), "cache_copy") #for debugging which files are actually used...
            # shutil.copy(cached_name, cache_copy_dir+"/" + cache_hash(key))
            return cached["value"]
    return None


def cache(key, value):
    if not os.path.exists(cache_dir):
        os.mkdir(cache_dir)
    cached_name = cache_dir + "/" + cache_hash(key)
    pickle.dump({"key": key, "value": value}, open(cached_name, "wb"))


tiktoken_encodings = {
    "gpt-4o": tiktoken.get_encoding("cl100k_base"),
    "gpt-4o-mini": tiktoken.get_encoding("cl100k_base"),
    "gpt-4-turbo": tiktoken.get_encoding("cl100k_base"),
    "gpt-4-turbo-preview": tiktoken.get_encoding("cl100k_base"),
    "gpt-4": tiktoken.get_encoding("cl100k_base"),
    "gpt-3.5-turbo": tiktoken.get_encoding("cl100k_base"),
    "gpt-3.5-turbo-instruct": tiktoken.get_encoding("cl100k_base"),
    "gpt-3.5-turbo-16k": tiktoken.get_encoding("cl100k_base"),
    "text-davinci-003": tiktoken.get_encoding("p50k_base"),
    "text-davinci-002": tiktoken.get_encoding("p50k_base"),
    "text-davinci-001": tiktoken.get_encoding("r50k_base"),
    "text-curie-001": tiktoken.get_encoding("r50k_base"),
    "text-babbage-001": tiktoken.get_encoding("r50k_base"),
    "text-ada-001": tiktoken.get_encoding("r50k_base"),
    "davinci": tiktoken.get_encoding("r50k_base"),
    "curie": tiktoken.get_encoding("r50k_base"),
    "babbage": tiktoken.get_encoding("r50k_base"),
    "ada": tiktoken.get_encoding("r50k_base"),
}

max_llm_context_length = {
    "gpt-4o": 128000,
    "gpt-4-turbo": 16384 * 2,
    "gpt-4-turbo-preview": 16384 * 2,
    "gpt-3.5-turbo-16k": 16384,
    "gpt-4": 8192,
    "gpt-3.5-turbo": 4096,
    "gpt-3.5-turbo-instruct": 4096,
    "text-davinci-003": 4096,
    "text-davinci-002": 4096,
    "text-davinci-001": 2049,
    "text-curie-001": 2049,
    "text-babbage-001": 2049,
    "text-ada-001": 2049,
    "davinci": 2049,
    "curie": 2049,
    "babbage": 2049,
    "ada": 2049
}


def is_chat_model(model):
    return ("gpt-4" in model) or ("gpt-3.5-turbo" in model) and ("gpt-3.5-turbo-instruct" not in model)


def token_overhead(model):
    if is_chat_model(model):
        return 300  # these models have some overhead because of the system message and chat structure
    return 0


def num_tokens_from_string(string: str, model: str) -> int:
    """Returns the number of tokens in a text string."""
    if not model in tiktoken_encodings:
        raise Exception(f"Tiktoken encoding unknown for LLM: {model}")
    encoding = tiktoken_encodings[model]
    num_tokens = len(encoding.encode(string))
    return num_tokens




# Queries an LLM for continuations of a batch of prompts given as a list
def query_LLM_batch(model, prompt_batch, max_tokens, use_cache=None, temperature=None,system_message=None,stop=None):
    # Print all arguments to check if any are None
    print("Arguments:")
    print(f"model: {model}")
    print(f"prompt_batch: {prompt_batch}")
    print(f"max_tokens: {max_tokens}")
    print(f"use_cache: {use_cache}")
    print(f"temperature: {temperature}")
    print(f"system_message: {system_message}")
    print(f"stop: {stop}")
    
    # Handle None for system_message
    if system_message is None:
        system_message = "You are a helpful assistant."
    
    # Handle None for stop - it should only be passed to the API if it's not None
    stop = stop if stop is not None else []
    
    
    global current_openai_model
    current_openai_model = model    #needed for the Aalto Azure GPT API callbacks

    if temperature is None:
        temperature=0 #by default, operate fully deterministically
        
    print("OK 1")
    
    
    # Check for None values in API_type, model, and prompt_batch elements
    print("Checking for None values in API_type, model, and prompt_batch...")

    if API_type is None:
        print("API_type is None")
    else:
        print(f"API_type: {API_type}")

    if model is None:
        print("model is None")
    else:
        print(f"model: {model}")

    # Check each element in prompt_batch for None
    for i, prompt in enumerate(prompt_batch):
        if prompt is None:
            print(f"prompt_batch element at index {i} is None")
        else:
            print(f"prompt_batch element at index {i}: {prompt}")
        
        
    if use_cache is None:
        use_cache=False
    cache_key=(API_type+"_"+model+"".join(prompt_batch)).encode('utf-8')
    if use_cache:
        cached_result=load_cached(cache_key)
        if cached_result is not None:
            return cached_result

    print("OK 2")
    start_time = time.time()

    #choose whether to use the chat API or the older query API
    if is_chat_model(model):
        if system_message is None:
            system_message = "You are a helpful assistant."
        if API_type=="Aalto" and (client_async is None):
            # In case no async API client, fall back to running the prompts one-by-one
            continuations=[]
            for prompt in prompt_batch:
                success=False
                while not success:
                    try:
                        response = client.chat.completions.create(
                            model=model,
                            # the model variable must be set, but has no effect, model selection done in update_base_url_for_aalto
                            messages=[
                                {"role": "system", "content": system_message},
                                {"role": "user", "content": prompt},
                            ],
                        )
                        success=True
                    except openai.RateLimitError:
                        print("Rate limit error! Will retry in 5 seconds")
                        time.sleep(5)
                    if response is None or response.choices is None:
                        print("No response from API, will retry in 5 seconds. Check VPN settings if you're not in Aalto intranet.")
                        time.sleep(5)
                        success=False
                if response.choices[0].message.content is None:
                    continuations.append("")
                else:
                    continuations.append(response.choices[0].message.content.strip())
        else:
                # each batch in the prompt becomes its own asynchronous chat completion request
            async def batch_request(prompt_batch):
                tasks=[]
                for prompt in prompt_batch:
                    messages = [
                        {"role": "system", "content": system_message},
                        {"role": "user", "content": prompt},
                    ]
                    tasks.append(client_async.chat.completions.create(
                        model=model,
                        messages=messages,
                        temperature=temperature,
                        max_tokens=max_tokens,
                        n=1,  # one completion per prompt
                        stop=stop,
                        frequency_penalty=0.0,
                        presence_penalty=0.0,
                    ))
                return await asyncio.gather(*tasks)

            loop = asyncio.get_event_loop()
            responses = loop.run_until_complete(batch_request(prompt_batch))
            continuations = [response.choices[0].message.content.strip() for response in responses]

        print("OK 3")
        # before we return the continuations, ensure that we don't violate OpenAI's rate limits
        total_tokens = 0
        for prompt in prompt_batch:
            total_tokens += num_tokens_from_string(string=system_message, model=model)
            total_tokens += num_tokens_from_string(string=prompt, model=model)
        for continuation in continuations:
            total_tokens += num_tokens_from_string(string=continuation, model=model)
        max_tokens_per_minute = 600000  # currently imposed limit for GPT-4o on tier 5 is 30 million TPM
        wait_seconds = (total_tokens / max_tokens_per_minute) * 60.0
        #print(f"Waiting {wait_seconds} seconds to ensure staying within rate limit")
        time_elapsed=time.time() - start_time
        if time_elapsed<wait_seconds:
            time.sleep(wait_seconds-time_elapsed)

    else:
        # The old completions API supports batched prompts out-of-the-box
        response = client.completions.create(
            model=model,
            prompt=prompt_batch,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            stop=stop,
            n=1  # one completion per prompt
        )
        # extract continuations
        continuations = [choice.text for choice in response.choices]

        # before we return the continuations, ensure that we don't violate OpenAI's rate limits
        total_tokens = 0
        for prompt in prompt_batch:
            total_tokens += num_tokens_from_string(string=prompt, model=model)
        for continuation in continuations:
            total_tokens += num_tokens_from_string(string=continuation, model=model)
        max_tokens_per_minute = 90000  # currently imposed limit for ChatGPT models
        wait_seconds = (total_tokens / max_tokens_per_minute) * 60.0
        time_elapsed=time.time() - start_time
        if time_elapsed<wait_seconds:
            time.sleep(wait_seconds-time_elapsed)

    if use_cache:
        cache(key=cache_key,value=continuations)
    return continuations

def query_LLM(prompts, model=None, max_tokens=None, use_cache=None, temperature=None, system_message=None,stop=None):
    """
        Query a Language Model (LLM) with one or more prompts.

        This function sends prompts to a specified language model and returns the generated continuations for each prompt in batches.

        Args:
            prompts (str or list of str): The prompt or list of prompts to be sent to the LLM. If a single string is provided, it will be wrapped in a list.
            model (str, optional): The identifier for the model to be queried. Defaults to "gpt-4o".
            max_tokens (int, optional): The maximum number of tokens to generate in each completion. If None, the model's default will be used.
            use_cache (bool, optional): Whether to use cached results if available. Defaults to False.
            temperature (float, optional): Sampling temperature to use, in range (0, 1]. Higher values mean the model will take more risks. Defaults to 0.
            system_message (str, optional): A system message that can influence the generated response. Defaults to None.
            stop (str or list of str, optional): Sequences where the model will stop producing further tokens. Defaults to None.

        Returns:
            list of str: The continuations generated by the model for each input prompt.

        Examples:
            >>> responses = query_LLM("What is the capital of France?")
            >>> print(responses)
            ["The capital of France is Paris."]

            >>> prompts = ["Translate the following sentence to French: 'Hello, world!'", "Translate the following sentence to Spanish: 'Good morning!'"]
            >>> responses = query_LLM(prompts, model="gpt-3.5-turbo", max_tokens=50, temperature=0.5)
            >>> print(responses)
            ["Bonjour, le monde!", "¡Buenos días!"]
        """
    if model is None:
        model="gpt-4o"
    if use_cache is None:
        use_cache=False
    if temperature is None:
        temperature=0
    return_single=False
    if isinstance(prompts, str):
        prompts=[prompts] #the following code always expects a list of prompts
        return_single=True

    #Query the LLM in batches
    continuations=[]
    batch_size = 10  # The exact max batch_size for each model is unknown. This seems to work for all, and provides a nice speed-up.
    N = len(prompts)
    print("N:",N)
    for i in range(0, N, batch_size):
        prompt_batch=prompts[i:min([N, i + batch_size])]
        print("prompt_batch for loop:", prompt_batch)

        continuations+=query_LLM_batch(model=model,
                                 prompt_batch=prompt_batch,
                                 max_tokens=max_tokens,
                                 use_cache=use_cache,
                                 temperature=temperature,
                                 system_message=system_message,
                                 stop=stop)
        print("continuations for loop:", continuations)

        if N>batch_size:
            print_progress_bar(min([N, i + batch_size]), N,printEnd="")
    return continuations[0] if return_single else continuations


## TODO: Integrate with other query_LLM function
def query_LLM_with_response_format(prompt, response_format, model="gpt-4o", max_tokens=None):
    """
    Queries an LLM with a specified prompt and returns the response parsed into the specified response format class.
    Useful for when you want a structured output.

    Args:
        prompt (str): The input text prompt for the model.
        response_format (Type[BaseModel]): The Pydantic model class to parse the response into.
        model (str, optional): The model to use for querying (default is 'gpt-4o').
        max_tokens (int, optional): Maximum number of tokens in the response (default is None).

    Returns:
        BaseModel: The parsed response as an instance of the specified Pydantic model class.
    """
    success = False
    while not success:
        try:
            response = client.beta.chat.completions.parse(
                model=model,
                messages=[
                    {"role": "user", "content": prompt},
                ],
                response_format=response_format,
                max_tokens=max_tokens,
            )
            success = True
        except openai.RateLimitError:
            print("Rate limit error! Will retry in 5 seconds")
            time.sleep(5)
    return response.choices[0].message.parsed


def set_cache_directory(dir):
    global cache_dir
    cache_dir=dir

def get_cache_directory():
    return cache_dir



def embed(texts,use_cache=None,model=None,verbose=True):

    if model is None:
        model = "text-embedding-ada-002"

    if use_cache is None:
        use_cache = True

    cache_key=(API_type+"_"+model+"".join(texts)).encode('utf-8')
    if use_cache:
        cached_result=load_cached(cache_key)
        if cached_result is not None:
            if verbose:
                print("Loaded embeddings from cache, hash", cache_hash(cache_key))
            return cached_result


    #query embeddings from the API
    texts=[json.dumps(s) for s in texts]  #make sure we escape quotes in a way compatible with GPT-3 API's internal use of json
    batch_size = 32
    N = len(texts)

    embed_matrix=[]
    for i in range(0, N, batch_size):
        print_progress_bar(i, N, printEnd="")
        embed_batch=texts[i:min([N, i + batch_size])]
        embeddings = embed_client.embeddings.create(input=embed_batch, model=model)
        for j in range(len(embed_batch)):
            embed_matrix.append(embeddings.data[j].embedding)
    print("")
    embed_matrix=np.array(embed_matrix)
    #dim = len(embeddings['data'][0]['embedding'])
    #embed_matrix = np.zeros([N, dim])
    #for i in range(N):
    #    embed_matrix[i, :] = embeddings['data'][i]['embedding']

    #update cache
    if use_cache:
        cache(cache_key,embed_matrix)

    #return results
    return embed_matrix


def reduce_embedding_dimensionality(embeddings,num_dimensions,method="UMAP",use_cache=True,n_neighbors=None,verbose=True):
    if isinstance(embeddings,list):
        #embeddings is a list of embedding matrices => pack all to one big matrix for joint dimensionality reduction
        all_emb = np.concatenate(embeddings, axis=0)
    else:
        all_emb = embeddings
    def unpack(x,embeddings_list):
        row = 0
        result = []
        for e in embeddings_list:
            N = e.shape[0]
            result.append(x[row:row + N])
            row += N
        return result

    cache_key=(str(all_emb.tostring())+str(num_dimensions)+method+str(n_neighbors)).encode('utf-8')
    if use_cache:
        cached_result=load_cached(cache_key)
        if cached_result is not None:
            if verbose:
                print("Loaded dimensionality reduction results from cache, hash ", cache_hash(cache_key))
            if isinstance(embeddings, list):
                return unpack(cached_result,embeddings)
            else:
                return cached_result
    from sklearn.manifold import MDS
    from sklearn.manifold import TSNE
    import umap
    from sklearn.decomposition import PCA
    #cosine distance
    all_emb=all_emb/np.linalg.norm(all_emb,axis=1,keepdims=True)

    if method=="MDS":
        mds=MDS(n_components=num_dimensions,dissimilarity="precomputed")
        cosine_sim = np.inner(all_emb, all_emb)
        cosine_dist = 1 - cosine_sim
        x=mds.fit_transform(cosine_dist)
    elif method=="TSNE":
        tsne=TSNE(n_components=num_dimensions)
        x=tsne.fit_transform(all_emb)
    elif method=="PCA":
        pca=PCA(n_components=num_dimensions)
        x=pca.fit_transform(all_emb)
    elif method=="UMAP":
        if n_neighbors is None:
            n_neighbors=5
        reducer = umap.UMAP(n_components=num_dimensions,metric='cosine',n_neighbors=n_neighbors)
        x=reducer.fit_transform(all_emb)
    else:
        raise Exception("Invalid dimensionality reduction method!")

    if use_cache:
        cache(cache_key,x)

    if isinstance(embeddings, list):
        return unpack(x,embeddings)
    return x


In [48]:
import random
import pandas as pd
import numpy as np
import re
from Levenshtein import distance
from collections import defaultdict
from rapidfuzz import fuzz

#Construct a coding prompt for a single text (unit of analysis)
def construct_prompt(coding_instruction,few_shot_texts,few_shot_codes,codes_so_far,text):
    # The prompt always starts with the user-defined coding instruction
    prompt = coding_instruction + "\n\n"

    # Next, we optionally add all the codes created so far, to allow code reuse instead of creating redundant
    # new and only slightly different codes for new coded texts
    if (codes_so_far is not None) and len(codes_so_far)>0:
        if len(codes_so_far)>0:
            prompt += "Examples of codes to use. Please add new codes when needed:\n"
            # Shuffle codes to mitigate LLM recency bias
            codes_so_far=codes_so_far.copy()
            random.shuffle(codes_so_far)
            # Add each code as a new line
            for code in codes_so_far:
                prompt+=code+"\n"

    # Next, we add the few-shot examples separated by ###
    # We first shuffle the examples to mitigate LLM recency bias
    l = list(zip(few_shot_texts, few_shot_codes))
    random.shuffle(l)
    few_shot_texts, few_shot_codes = zip(*l)
    for i in range(len(few_shot_texts)):
        prompt+="\n\n###\n\nText: "+few_shot_texts[i]+"\n\n"+"Codes: "+few_shot_codes[i]

    # Finally, we add the text to code, suggesting the LLM that it should continue the prompt with the codes
    prompt+="\n\n###\n\nText: "+text+"\n\n"+"Codes:"
    return prompt


# Extract a string of codes (separated by semicolons) from LLM continuation
def continuation_to_code_string(continuation):
    stopseq="###"
    continuation = continuation.split(stopseq)[
        0]  # if the model started generating new texts, only keep the continuation
    continuation = continuation.lstrip(" \n")  # strip leading spaces and newlines
    continuation = continuation.rstrip(" ;\n")  # strip trailing spaces, semicolons and newlines
    return continuation






# Convert from a string representation of multiple codes to a list
def codes_to_string(codes):
    return "; ".join(codes)

# Convert list of codes to a single string
def string_to_codes(s):
    codes = s.split(";")
    codes = [t.strip() for t in codes]
    return codes

#Code a list of texts using an instruction string and few-shot examples of texts and codes
def code_texts(coding_instruction,
               few_shot_texts,
               few_shot_codes,
               gpt_model,
               texts,
               use_cache=True,
               verbose=False,
               max_tokens=None):
    if max_tokens is None:
        max_tokens=64 #we assume the codes are short. TODO: measure the token length of few-shot examples, adjust this accordingly (e.g., 2x)

    #If we don't care about code consistency, we can speed things up by coding batches of texts in parallel.
    #First, we construct the prompts for all coded texts
    prompts=[]
    for text in texts:
        prompts.append(construct_prompt(coding_instruction=coding_instruction,
                            few_shot_texts=few_shot_texts,
                            few_shot_codes=few_shot_codes,
                            codes_so_far=None,
                            text=text))
    if verbose:
        print(f"Constructed {len(prompts)} prompts, example: {prompts[-1]}")

    #Query the LLM
    continuations=query_LLM(model=gpt_model,
                            prompts=prompts,
                            max_tokens=max_tokens,
                            use_cache=use_cache)

    # Extract code strings from the LLM continuations
    result_codes=[continuation_to_code_string(c) for c in continuations]

    # Ensure that the few-shot examples got coded correctly
    for i,text in enumerate(texts):
        if text in few_shot_texts:
            few_shot_code=few_shot_codes[few_shot_texts.index(text)]
            print(f"Replacing code {result_codes[i]} with {few_shot_code}")
            result_codes[i]=few_shot_code.strip()

    return result_codes



def extract_single_codes(df,multiple_column="codes",single_column="code",count_column="count",id_column="code_id",merge_singular_and_plural=False):
    df=df.copy()  #will be modified to contain the contents
    # Parse individual codes
    N = df.shape[0]
    df_append = None  # replicated rows are collected in this df
    df[single_column]=df[multiple_column].copy()
    for i in range(N):
        codes = df.at[i, multiple_column]
        codes = codes.split(";")
        codes = [t.lstrip() for t in codes]

        # if there's more than a single topic, create copies of the entry
        if len(codes) > 1:
            row = df.loc[i:i].copy()  # make a temp df of the current row
            df.at[i, multiple_column] = "PRUNED"  # mark the current row for pruning
            for code in codes:
                row[single_column] = code
                # display(row)
                if df_append is None:
                    df_append = row.copy()
                else:
                    df_append = pd.concat([df_append,row.copy()],axis=0)
    df = df[df[multiple_column] != "PRUNED"]
    df = pd.concat([df, df_append], axis=0, ignore_index=True)
    df.reset_index(drop=True, inplace=True)
    codes = df[single_column].unique().tolist()
    #print(f"Added {df.shape[0] - N} duplicate text rows to handle texts with multiple codes")
    #df=df.drop(columns=[multiple_column])

    # Merge singular and plural forms of the same code
    if merge_singular_and_plural:
        for code in codes:
            if code + "s" in codes:
                df.loc[df[single_column] == code, single_column] = code + "s"
                codes.remove(code)

    # Count and sort to have the most frequent topics at the top
    df[count_column] = 0
    counts = []
    for code in codes:
        count = df[df[single_column] == code].shape[0]
        counts.append(count)
        df.loc[df[single_column] == code, count_column] = count

    df.sort_values(by=[count_column, single_column], inplace=True, ascending=False)
    return df



def group_codes_using_embeddings(df,embedding_context="",embedding_model="text-similarity-curie-001",min_group_size=3,max_group_emb_codes=100,group_desc_codes=2,group_desc_freq=True,grouping_dim=5,use_cache=True,dimred_method=None):
    """
    Combines codes to groups/themes using HDBSCAN clustering of GPT-3 embedding vectors for each code.

    Also counts how many coded texts were assigned with each code and group. Note that one text can be assigned to
    multiple codes and therefore multiple groups.

    Args:
        df                  A dataframe with a "text_id" and "codes" column with codes separated by semicolons (as
                            produced by the code() function), or a "code" column with single codes (as produced by
                            the extract_single_codes() function).
        embedding_context   A context string appended to each code when computing code embeddings. This may help
                            disambiguating code labels that might be synonyms in the general case, but have different
                            meaning in the coding context. E.g., ", in context of experiencing games as art"
        group_desc_codes    How many of the most frequent codes to use for describing each code group
        group_desc_freq     Whether to use the group frequency in the group description
        min_group_size      Minimum number of codes in each code group
        use_cache           If True, skips OpenAI API calls if results found in the cache
                            (see set_cache_directory)

    Returns:
        A dictionary of results with the following keys:
        "coded"             Same as the input DataFrame but with the following additional columns:
                            "text_id", "text", "codes", "code", "code_id", "code_count", "group_codes", "group_id",
                            "group_count", "group_desc" columns. "codes" is a comma-separated list of all codes
                            assigned to each text. "code" is a single code, and each text is duplicated to as many
                            DataFrame rows as there are codes assigned to that text. code and group id:s are integers
                            that can be used to index the embedding arrays (see below). "text_id" is an index to the
                            original list of texts.
        "embeddings"        A numpy array of shape [num_codes,num_embedding_dimensions]
        "embeddings_5d"     Dimensionality-reduced embeddings used for clustering
        "group_embeddings"  A numpy array of shape [num_groups,num_embedding_dimensions], with the (normalized)
                            count-weighted average of each group's embedding vector. Can be useful for finding the
                            closest code groups between two separately coded datasets
        "group_code_ids"    list of lists of code ids (indices) for each group. Useful for, e.g., indexing the
                            embeddings of each group

        Note that if you want to view or list subsets of the results, e.g., the codes for each original text,
        this is easy to do by using Pandas drop_duplicates() on a suitable column such as "text_id"
    """
    if dimred_method is None:
        dimred_method="UMAP"

    df=df.copy() #will be modified to contain the results

    # extract individual codes
    if "text_id" not in df.columns:
        raise Exception("The input DataFrame does not have a \"text_id\" column!")
    if "code" in df.columns:
        print("The input dataframe already has a \"code\" column => skipping the initial extraction of single codes.")
    else:
        if "codes" not in df.columns:
            raise Exception("The input DataFrame does not have a \"codes\" column!")
        df["codes"] = df["codes"].astype(str)
        df = extract_single_codes(df)

    codes = df["code"].unique().tolist()

    code_counts=df.drop_duplicates(subset=["code"])["count"].to_numpy()
    n_codes=len(codes)
    n_texts=df["text_id"].nunique()

    #compute code embeddings
    print("Calculating code embeddings...")
    codes_with_context=[code + embedding_context for code in codes]
    embeddings=embed(codes_with_context,model=embedding_model,use_cache=use_cache)
    embed_dim=embeddings.shape[1]


    print("Reducing embedding dimensionality to 2D for visualization")
    dimred_neighbors=min_group_size+2 #heuristic: for the grouping to work, dimensionality reduction needs to consider a somewhat larger neighborhood
    embeddings_2d = reduce_embedding_dimensionality(embeddings,2,use_cache=use_cache,method=dimred_method,n_neighbors=dimred_neighbors)

    #reduce dimensionality
    print("Reducing embedding dimensionality for grouping...")
    embeddings_reduced=reduce_embedding_dimensionality(embeddings, grouping_dim, method=dimred_method,use_cache=use_cache,n_neighbors=dimred_neighbors)

    #group (UMAP + HDBSCAN)
    print("Grouping...")
    clustering_method='hdbscan'

    if clustering_method=="hdbscan":
         import hdbscan
         cluster = hdbscan.HDBSCAN(min_cluster_size=min_group_size,
                                   metric='euclidean',
                                   cluster_selection_method='eom').fit(embeddings_reduced)
         if np.min(cluster.labels_)<0:
             has_outliers=True
             labels=cluster.labels_+1  #+1 because the outlier cluster has id -1, which we now map to 0 to make indexing easier
         else:
             has_outliers = False
             labels=cluster.labels_

    elif clustering_method=="k-means":
         from sklearn.cluster import KMeans

         n_groups = 20
         kmeans = KMeans(init='k-means++', n_clusters=n_groups, n_init=20)
         kmeans.fit(embeddings_reduced)

         # Predict the cluster for all the samples
         labels = kmeans.predict(embeddings_reduced)

    #Update the cluster_id column
    n_groups = np.max(labels) + 1
    print(f"Combined {len(codes)} codes to {n_groups} groups")
    groups= [''] * n_groups
    group_code_ids=[None] * n_groups
    for i in range(n_groups):
        group_code_ids[i]=[]

    df["group_id"]=0
    df["code_id"]=-1
    df["code_2d_0"]=0
    df["code_2d_1"]=0
    for code_id,code in enumerate(codes):
         #group label for this code
         group_id=labels[code_id]
         group_code_ids[group_id].append(code_id)
         if groups[group_id]=='':
              groups[group_id]=codes[code_id]
         else:
              groups[group_id]+=', '+codes[code_id]
         df.loc[df["code"] == code,"group_id"]=group_id
         df.loc[df["code"] == code,"code_id"]=code_id
         df.loc[df["code"] == code,"code_2d_0"]=embeddings_2d[code_id,0]
         df.loc[df["code"] == code,"code_2d_1"]=embeddings_2d[code_id,1]

    #Update the various group stats and descriptors
    df["group_codes"]=""
    df["group_count"]=0
    df["group_desc"]=""
    group_embeddings=np.zeros([n_groups,embed_dim])
    for i in range(n_groups):
        #update group codes and count
        rows=df["group_id"]==i
        df.loc[rows, "group_codes"]=groups[i]
        group_count=df.loc[rows,"text_id"].nunique()
        group_freq=round(100.0 * group_count / n_texts)
        df.loc[rows,"group_count"]=group_count
        df.loc[rows,"group_freq"]=group_freq
        df.loc[rows,"group_code_ids"]=",".join([str(code_id) for code_id in group_code_ids[i]])

        #group embeddings as weighted average of code embeddings
        code_ids=group_code_ids[i]
        if len(code_ids)>max_group_emb_codes:
            code_ids=code_ids[:max_group_emb_codes]
        weights=code_counts[code_ids]
        group_embeddings[i]=np.average(embeddings[code_ids],axis=0,weights=weights)
        group_embeddings[i]/=np.linalg.norm(group_embeddings[i])+1e-10

        #codes sorted by distance from the group embedding
        all_code_embeddings=embeddings[group_code_ids[i]]
        dist=np.inner(all_code_embeddings,group_embeddings[i].reshape([1,-1]))[:,0]
        dist=1.0-dist
        sorted_group_code_ids=np.array(group_code_ids[i])[np.argsort(dist)]
        sorted_group_codes=np.array(codes)[sorted_group_code_ids]
        df.loc[rows,"sorted_group_codes"]=",".join(sorted_group_codes)

        # Helper for naming a group based on most frequent codes
        def group_name_freq(codes, freq):
            codes = codes.split(',')
            result = codes[0]
            if len(codes) > 1:
                for code in codes[1:min([len(codes), group_desc_codes])]:
                    result += ", " + code
            if group_desc_freq:
                result+=f" ({freq}%)"
            return result
        df.loc[rows, "group_desc"]=group_name_freq(groups[i],group_freq)

    #Mark outliers if using hdbscan
    outliers=None
    df["outlier"]=0
    df["not_outlier"]=1 #just to make sorting easier
    if clustering_method=="hdbscan" and has_outliers:
        df.loc[df["group_id"]==0,"outlier"]=1
        df["not_outlier"] = 1-df["outlier"]
        '''
        outliers=df[df["group_id"]==0]
        df=df[df["group_id"]!=0]
        df["group_id"]=df["group_id"]-1
        group_code_ids=group_code_ids[1:]
        group_embeddings=group_embeddings[1:]
        n_groups-=1
        '''

    #Sort
    df.sort_values(by=["not_outlier","group_count","group_codes","count", "code"], inplace=True, ascending=False)

    return {"df":df,"embeddings":embeddings,"embeddings_reduced":embeddings_reduced,"embeddings_2d":embeddings_2d,"group_embeddings":group_embeddings,"group_code_ids":group_code_ids,"n_groups":n_groups,"code_counts":code_counts}


def code_df(df,
           column_to_code,
           coding_model,
           embedding_model,
           embedding_context,
           dimred_method,
           dimred_neighbors,
           use_cache=True,
           verbose=False,
           pruned_code=None):
    #Code
    print(f"Coding the {column_to_code} column of the dataframe")
    texts = df[column_to_code].astype(str).tolist()
    few_shot_codes = df.loc[df['use_as_example'] == 1, 'human_codes'].astype(str).tolist()
    few_shot_texts = df.loc[df['use_as_example'] == 1, column_to_code].astype(str).tolist()

    # check if the df specifies the instructions
    if "coding_instructions" in df:
        coding_instruction = df["coding_instructions"][0]
    else:
        raise Exception(
            "Coding instructions not specified. Please specify using the first row of a column named \"coding_instructions\"")

    codes = code_texts(coding_instruction=coding_instruction,
                       few_shot_texts=few_shot_texts,
                       few_shot_codes=few_shot_codes,
                       gpt_model=coding_model,
                       texts=texts,
                       use_cache=use_cache,
                       verbose=verbose)
    df_coded=df.copy()
    df_coded["codes"]=codes
    df_coded["text_id"]=list(range(0, len(texts)))  #for keeping track of which original texts the codes refer to

    print("\nCoding instruction:\n\n")
    print(coding_instruction)

    print("Coded data:")
    df_coded.reset_index(drop=True,inplace=True) #for cleaner printouts
    print(df_coded[[column_to_code, "codes"]].head(20))

    # Compare gpt and human codes
    print("Comparing LLM and human codes based on modified Hausdorff distance in embedding space...")
    df_cmp=df_coded[df_coded["human_codes"].notnull()].copy()
    df_cmp=df_cmp[df_cmp["human_codes"]!=""]
    df_cmp=df_cmp[df_cmp["use_as_example"]!="1"]
    df_cmp=df_cmp[[column_to_code,"human_codes","codes"]]
    df_cmp=gpt_human_code_dist(df=df_cmp,embedding_context=embedding_context,embedding_model=embedding_model)

    # Create a df with a line per code, sorted by code frequencies, with original texts/groundings in other columns
    print("Formatting output...")
    # First, construct a df with a line code, duplicated for each coded text
    df_codes = df_coded.copy()
    df_codes["codes"] = df_codes["codes"].astype(str)
    df_codes = extract_single_codes(df_codes)
    df_codes = df_codes[["code","count","text_id",column_to_code]]
    max_count=df_codes["count"].max()

    # Now, pack the duplicate rows into columns, and convert from single text id:s to lists of text ids for each code
    df_single = df_codes.drop_duplicates(subset=["code"]).copy()
    text_ids_all=[]
    for i in range(max_count):
        df_single[f"text {i}"]=None #placeholder for the texts

    for code in df_single["code"]:
        rows=df_codes[df_codes["code"]==code]
        text_ids=rows["text_id"].astype(str).to_list()
        texts=rows[column_to_code].astype(str).to_list()
        for i in range(len(texts)):
            df_single.loc[df_single["code"] == code, f"text {i}"]=f"{text_ids[i]}: {texts[i]}"
        text_ids_all.append(",".join(text_ids))
    df_single["text_ids"]=text_ids_all
    df_single=df_single.drop(columns=["text_id",column_to_code])
    df_single.reset_index(drop=True,inplace=True)
    df_codes=df_single

    # Create and print a summary of codes
    df_code_summary=df_codes[["code","count","text 0"]].rename(columns={"text 0":"example text"})
    print("Total number of codes: ",df_single.shape[0])
    print("\nCodes sorted by number of groundings:")
    print(df_code_summary.head(df_code_summary.shape[0]))

    # Calculate 2d embeddings for each code, for visualizing
    print("Embedding codes and reducing dimensionality for visualization...")
    codes_with_context=[code + embedding_context for code in df_codes["code"].astype(str).tolist()]
    embeddings=embed(codes_with_context,model=embedding_model,use_cache=use_cache)
    embeddings_2d = reduce_embedding_dimensionality(embeddings=embeddings,
                                                    num_dimensions=2,
                                                    use_cache=use_cache,
                                                    method=dimred_method,
                                                    n_neighbors=dimred_neighbors)
    df_codes["code_2d_0"]=embeddings_2d[:,0]
    df_codes["code_2d_1"]=embeddings_2d[:,1]


    # Prune codes, if a pruning code specified
    df_coded_pruned=None
    df_codes_pruned=None
    if pruned_code is not None:
        df_coded_pruned=df_coded[~df_coded["codes"].str.contains(pruned_code)].copy()
        df_coded_pruned.reset_index(drop=True, inplace=True)
        df_codes_pruned=df_codes[~df_codes["code"].str.contains(pruned_code)].copy()
        df_codes_pruned.reset_index(drop=True, inplace=True)

    # Return results as a dict
    result={}
    result["prompt"]=construct_prompt(
        coding_instruction=coding_instruction,
        few_shot_texts=few_shot_texts,
        few_shot_codes=few_shot_codes,
        codes_so_far=None,
        text=texts[0]
    )
    result["df_coded"]=df_coded
    result["df_codes"]=df_codes
    result["df_coded_pruned"]=df_coded_pruned
    result["df_codes_pruned"]=df_codes_pruned
    result["df_validate"]=df_cmp
    #result["df_code_summary"]=df_code_summary
    return result

def group_codes(
    df_codes,
    df_data,
    grouping_model,
    use_cache,
    random_seed=None,
    verbose=False):

    #extract and check instructions
    theme_elicitation_instructions=df_data["theme_elicitation_instructions"][0]
    if not "<codes>" in theme_elicitation_instructions:
        raise Exception("Theme elicitation instructions missing the <codes> placeholder which will be replaced with the list of codes in the LLM prompt.")

    code_grouping_instructions=df_data["code_grouping_instructions"][0]
    if not "<codes>" in code_grouping_instructions:
        raise Exception("Code grouping instructions missing the <codes> placeholder which will be replaced with the list of codes in the LLM prompt.")
    if not "<themes>" in code_grouping_instructions:
        raise Exception("Code grouping instructions missing the <themes> placeholder which will be replaced with the list of themes in the LLM prompt.")

    #STEP 1: Elicit a list of themes
    #If there are more codes than fit a single prompt, we elicit the themes based on a random subset.
    codes=df_codes["code"].astype(str).to_list()
    random.seed(random_seed)
    random.shuffle(codes)

    #Determine how many codes we can fit the prompt
    code_tokens=[num_tokens_from_string(code,grouping_model) for code in codes]
    overhead=num_tokens_from_string(string=theme_elicitation_instructions,model=grouping_model)\
             +token_overhead(grouping_model)+1000 #token overhead for theme list, system message and prompt start
    max_tokens_for_codes=max_llm_context_length[grouping_model]-overhead
    code_tokens_cumulative=0
    max_codes=0
    for i in range(len(code_tokens)):
        code_tokens_cumulative+=code_tokens[i]+1 #+1 because of newlines after each code in the prompt
        if code_tokens_cumulative>max_tokens_for_codes:
            break
        max_codes=i

    #Prompt the themes
    print(f"Identifying themes based on {max_codes} codes...")
    codes_as_string="\n".join(codes[:max_codes])
    prompt=theme_elicitation_instructions
    prompt=prompt.replace("<codes>",codes_as_string)
    response=query_LLM(model=grouping_model,
                       prompts=[prompt],
                       temperature=0,
                       use_cache=use_cache)[0]
    if verbose:
        print("Prompt:\n")
        print(prompt+"\n")

    #Parse response
    response=response[response.find("{"):response.find("}")+1] #extract the .json part
    def correct_gpt_quotation_escape(s):
        return s #s.replace('\\\"\"','\\\"')       #Aalto gpt-3.5-turbo sometimes does a "double" escape of quotation marks
    response=correct_gpt_quotation_escape(response)
    j = json.loads(response)
    theme_list=j["themes"]
    theme_list=list(set(theme_list))  #remove possible duplicates
    other_theme="Other" #the theme identification probably misses some minor themes => they will get grouped under this theme
    theme_list.append(other_theme)
    themes_as_string="\n".join(theme_list)
    print("Themes identified:\n")
    print(themes_as_string+"\n")

    #STEP 2: Group codes under the themes in batches
    #prompt the LLM
    batch_size=50
    remaining_codes=codes.copy()
    themes={}   #results will be added to this dict, with codes as keys
    while len(remaining_codes)>0:
        current_remaining=len(remaining_codes)
        prompt=code_grouping_instructions
        code_batch=remaining_codes[:min(batch_size,len(remaining_codes))]
        print(f"Grouping a batch of {len(code_batch)} codes, total {current_remaining} remaining...")
        codes_as_string="\n".join(code_batch)
        if verbose:
            print("Code batch:")
            print(codes_as_string)
        prompt=prompt.replace("<codes>",codes_as_string)
        prompt=prompt.replace("<themes>",themes_as_string)
        response=query_LLM(model=grouping_model,
                           prompts=[prompt],
                           use_cache=use_cache)[0]
        response=response[response.find("{"):response.find("}")+1] #extract the .json part
        response = correct_gpt_quotation_escape(response)
        try:
            j = json.loads(response)
        except json.decoder.JSONDecodeError:
            print(".json parse of the following response failed:\n")
            print(response)
            exit()
        #remove codes that were successfully grouped
        for code in code_batch:
            if code in j:
                remaining_codes.remove(code)
                themes[code]=j[code]

        #prevent getting stuck in an infinite loop in case the LLM keeps making mistakes
        if len(remaining_codes)==current_remaining:
            print("These remaining codes could not be grouped, categorizing them as 'other':")
            for code in remaining_codes:
                print(code)
                themes[code]="Other"
            break

    #Add themes to the codes dataframe
    df_grouped=df_codes.copy()
    df_grouped.reset_index(drop=True, inplace=True)
    codes_all=df_grouped["code"]
    themes_all=[themes[code] for code in codes_all]
    df_grouped["theme"]=themes_all

    #Calculate theme counts
    themes_list=df_grouped["theme"].unique()
    df_grouped["theme_count"]=0
    df_grouped["theme_index"]=0
    for i,theme in enumerate(themes_list):
        df_theme=df_grouped[df_grouped["theme"]==theme]
        df_grouped.loc[df_grouped["theme"]==theme,"theme_count"]=df_theme["count"].sum()
        df_grouped.loc[df_grouped["theme"] == theme, "theme_index"]=i

    #Sort by both theme counts, themes, code_counts
    df_grouped.sort_values(by=["theme_count", "theme","count"], inplace=True, ascending=False)
    df_grouped.reset_index(drop=True, inplace=True)

    '''
    #Replace many very small themes with "Other"
    if other_threshold is not None:
        df_other=df_grouped[df_grouped["theme_count"]<=other_threshold].copy()
        df_rest=df_grouped[df_grouped["theme_count"]>other_threshold].copy()
        df_other["theme"]="Other"
        df_other["theme_count"]=df_other["count"].sum()
        df_grouped=pd.concat([df_rest,df_other],axis=0)
    '''

    #Some final formatting
    df_grouped.rename(columns={"count":"code_count"},inplace=True)
    cols=list(df_grouped.columns.values)
    cols.remove("theme")
    cols.remove("theme_count")
    if "index" in cols:
        cols.remove("index")
    if "level_0" in cols:
        cols.remove("level_0")
    df_grouped=df_grouped[["theme","theme_count"]+cols]
    return df_grouped

def code_and_group(df,
                   column_to_code,
                   coding_model,
                   embedding_model,
                   embedding_context=None,
                   min_group_size=3,
                   grouping_dim=5,
                   use_cache=True,
                   verbose=False,
                   dimred_method=None,
                   pruned_code=None):
    if embedding_context is None:
        embedding_context=""



    # Group codes
    group_info = group_codes(df_coded,
                             embedding_context=embedding_context,
                             embedding_model=embedding_model,
                             min_group_size=min_group_size,
                             use_cache=use_cache,
                             group_desc_codes=3,
                             grouping_dim=grouping_dim,
                             group_desc_freq=False,
                             dimred_method=dimred_method)

    # Create and print a summary of groups
    df_grouped=group_info["df"].copy()
    df_grouped = df_grouped.drop_duplicates(subset=["group_codes"])  # only print one text per group
    df_grouped.reset_index(drop=True,inplace=True) #for cleaner printouts
    df_group_summary=df_grouped[[column_to_code, "group_desc","group_freq"]].copy()
    print("\nCode groups sorted by group frequency")
    print(df_group_summary.head(20))

    # Convert the text-per-row grouped df to code-per-row,
    # which allows easier editing of the results in a spreadsheet editor

    # First, create an empty dataframe with correct columns
    columns=["group_desc","code"]
    for i in range(group_info["df"]["count"].max()):
        columns.append(f"text_{i+1}")
    df_editable=None #pd.DataFrame(columns=columns)

    # loop over groups
    groups = group_info["df"]["group_id"].unique()
    for group_id in groups:
        # select this group's rows
        group_data=group_info["df"][group_info["df"]["group_id"]==group_id]
        # skip if this is the outlier group produced by HDBSCAN
        if group_data["outlier"].iloc[0]==0:
            # loop over codes
            codes=group_data["code"].unique()
            for c in codes:
                code_data=group_data[group_data["code"]==c]
                # add a line for the code
                new_row={}
                new_row["group_desc"]=code_data["group_desc"].iloc[0]
                new_row["group_count"]=code_data["group_count"].iloc[0]
                new_row["code"]=c
                new_row["code_count"]=int(code_data["count"].iloc[0])
                new_row["code_2d_0"]=code_data["code_2d_0"].iloc[0]
                new_row["code_2d_1"] = code_data["code_2d_1"].iloc[0]

                for c in range(code_data.shape[0]):
                    new_row[f"text_{c+1}"]=code_data[column_to_code].iloc[c]
                new_row=pd.Series(data=new_row).to_frame().T
                if df_editable is None:
                    df_editable=new_row
                else:
                    df_editable=pd.concat(objs=[df_editable,new_row],axis=0,ignore_index=True)
    df_editable.reset_index(drop=True, inplace=True)

    # Return a dict with all the info the caller might need for further analysis or data export
    result=group_info.copy()
    result["prompt"]=construct_prompt(
        coding_instruction=coding_instruction,
        few_shot_texts=few_shot_texts,
        few_shot_codes=few_shot_codes,
        codes_so_far=None,
        text=texts[0]
    )
    result["df_coded"]=df_coded_not_pruned
    result["df_validate"]=df_cmp
    result["df_editable"]=df_editable
    result["df_code_summary"]=df_code_summary
    result["df_group_summary"]=df_group_summary
    return result



def code_and_embed(prompt_base,df,coding_model="text-curie-001",embedding_model="text-similarity-curie-001",column_to_code="texts",embedding_context="",use_cache=True,verbose=True):
    df_coded = code(prompt_base=prompt_base,
                      gpt_model=coding_model,
                      df=df,
                      column_to_code=column_to_code,
                      use_cache=use_cache,
                      verbose=verbose)
    df_single = extract_single_codes(df_coded)
    codes = df_single["code"].unique().tolist()
    code_counts=df_single.drop_duplicates(subset=["code"])["count"].to_numpy()
    codes_with_context=[code + embedding_context for code in codes]
    embeddings=embed(codes_with_context,model=embedding_model,use_cache=use_cache)
    return {"codes":codes,"counts":code_counts,"embeddings":embeddings}


def code_inductively(texts,
                     research_question,
                     few_shot_examples,
                     gpt_model,
                     use_cache=True,
                     max_tokens=None,
                     verbose=False):
    if max_tokens is None:
        # Set max_tokens dynamically based on maximum text length
        # Note that len(text) is in characters, not tokens
        max_tokens = max(max(len(text) for text in texts), 300)
    
    # Code batches of texts in parallel
    prompts = [construct_inductive_prompt(text, research_question, few_shot_examples)
               for text in texts]

    # Query the LLM
    continuations = query_LLM(
        model=gpt_model,
        prompts=prompts,
        max_tokens=max_tokens,
        use_cache=use_cache
    )

    # Attempt to correct any LLM formatting errors
    coded_texts = correct_coding_errors(texts, continuations, verbose=verbose)

    return coded_texts


def code_inductively_with_code_consistency(texts,
                                           research_question,
                                           few_shot_examples,
                                           gpt_model,
                                           use_cache=True,
                                           max_tokens=None,
                                           verbose=False):
    if max_tokens is None:
        # Set max_tokens dynamically based on maximum text length
        # Note that len(text) is in characters, not tokens
        max_tokens = max(max(len(text) for text in texts), 300)

    # Process texts sequentially to enforce code consistency
    coded_texts = []
    code_descriptions = {}
    for idx, text in enumerate(texts):
        # Update progress
        print_progress_bar(idx + 1, len(texts), printEnd="")

        # Find insights independent of existing codes
        insight_prompt = construct_insight_prompt(text, research_question)
        print("\n\ninsight_prompt:\n", insight_prompt)
        
        continuations = query_LLM(
            model=gpt_model,
            prompts=[insight_prompt],
            max_tokens=200,
            use_cache=use_cache
        )
        print("\n\ncontinuations:\n", continuations)

        insights = continuations[0]

        # Construct prompt, including a list of existing codes
        prompt = construct_inductive_prompt(
            text=text, 
            research_question=research_question,
            few_shot_examples=few_shot_examples,
            code_descriptions=code_descriptions,
            insights=insights
        )
        print("\n\prompt:\n", prompt)

        # Query the LLM
        continuations = query_LLM(
            model=gpt_model,
            prompts=[prompt],
            max_tokens=max_tokens,
            use_cache=use_cache
        )
        
        # Attempt to correct any LLM formatting errors
        coded_text_batch = correct_coding_errors([text], continuations, verbose=verbose)

        # Add singular coded text to output
        assert len(coded_text_batch) == 1
        coded_text = coded_text_batch[0]
        coded_texts.append(coded_text)

        # For any new codes, generate description and store
        for highlight, code in parse_codes(coded_text):
            if code not in code_descriptions:
                code_descriptions[code] = generate_code_description(
                    code,
                    [highlight],
                    research_question,
                    gpt_model,
                    use_cache
                )
                
    return coded_texts, code_descriptions


def code_deductively(texts,
                     research_question,
                     codebook,
                     gpt_model,
                     few_shot_examples=None,
                     use_cache=True,
                     verbose=False):
    """
    Apply deductive qualitative coding to a list of texts using a language model.

    This function uses a predefined codebook and prompts a GPT-based model to 
    generate coded versions of the input texts. It processes the texts deductively, 
    meaning the coding is guided by the provided codebook. Optionally, few-shot 
    examples can be included to guide the model's responses. The function returns 
    the coded texts and a dictionary of codes with corresponding highlights.

    Args:
        texts (list of str): A list of texts to be coded.
        research_question (str): The research question used to guide the language model.
        codebook (list of str or tuple): A list of code strings or (code, description) tuples representing the codebook.
        gpt_model (str): The identifier for the GPT model to be used for coding.
        few_shot_examples (pd.DataFrame, optional): DataFrame containing few-shot examples to guide the model. 
            Each row should contain a 'coded_text' column with pre-coded examples.
        use_cache (bool, optional): Whether to use cached model results if available. Defaults to True.
        verbose (bool, optional): Whether to print warnings for coding inconsistencies and errors. Defaults to False.

    Returns:
        tuple: 
            - coded_texts (list of str): The input texts with codes inserted by the language model.
    """
    # Set max_tokens dynamically based on maximum text length
    # Note that max_text_len is in characters, not tokens
    max_text_len = max(len(text) for text in texts)

    # Make codebook into a list of tuples if list of strings
    codebook = [(item,) if isinstance(item, str) else item for item in codebook]

    # Make sure codes in few_shot_examples are from codebook
    codebook_codes = [item[0] for item in codebook]
    if few_shot_examples is not None:
        for _, row in few_shot_examples.iterrows():
            for _, code in parse_codes(row.coded_text):
                if code not in codebook_codes:
                    print(f"WARNING: Few-shot examples contain code \"{code}\" that is not in the codebook")

    # Query the LLM
    prompts = [construct_deductive_prompt(text, research_question, codebook, few_shot_examples)
               for text in texts]
    continuations = query_LLM(
        model=gpt_model,
        prompts=prompts,
        max_tokens=max_text_len,
        use_cache=use_cache
    )

    # Attempt to correct any LLM formatting errors
    coded_texts = correct_coding_errors(texts, continuations, verbose=verbose)

    # Check output for any hallucinated codes
    for idx in range(len(coded_texts)):
        hallucinated_codes = []
        for _, code in parse_codes(coded_texts[idx]):
            if code not in codebook_codes:
                hallucinated_codes.append(code)
                if verbose:
                    print(f"WARNING: Output contains code \"{code}\" that is not in the codebook, removing...")

        # Remove any hallucinated codes
        for code in hallucinated_codes:
            coded_texts[idx] = _remove_code(coded_texts[idx], code)

    return coded_texts


def _remove_code(text, code_to_remove):
    highlight_pattern = r"(\*\*(.*?)\*\*<sup>(.*?)<\/sup>)"
    for full_match, highlight, codes in re.findall(highlight_pattern, text):
        codes = [code.strip() for code in codes.split(";")]
        if code_to_remove in codes:
            if len(codes) == 1:
                # If it's the only code, remove the entire highlighted structure
                new_text = highlight
            else:
                # Remove the specified code and reformat
                codes = [code for code in codes if code != code_to_remove]
                new_text = "**{}**<sup>{}</sup>".format(highlight, "; ".join(codes))
            
            # Replace the first occurrence of the full match with the modified text
            text = text.replace(full_match, new_text, 1)
            break  # Only remove one occurrence per function call
    return text


def construct_inductive_prompt(text,
                               research_question,
                               few_shot_examples,
                               code_descriptions=None,
                               insights=None,
                               ):
    # Optionally add guidance about given main insights into the prompt
    insight_text = " and the main insights from the text" if insights is not None else ""

    prompt = f"You are an expert qualitative researcher conducting a project with the research question: {research_question}. "
    prompt = f"""You are given a text to code inductively{insight_text}. Please carry out the following task:
- Respond by repeating the original text, but highlighting the coded statements by surrounding the statements with double asterisks, as if they were bolded text in a Markdown document.
- Include the associated code(s) immediately after the statement, separated by a semicolon and enclosed in <sup></sup> tags, as if they were superscript text in a Markdown document.
- Ignore text that is not insightful with regards to the research question.
- Preserve exact formatting of the original text. Do not correct typos or remove unnecessary spaces.\n\n"""

    # Optionally add existing codes into the prompt, to encourage consistency
    if code_descriptions is not None and len(code_descriptions) > 0:
        prompt += "Some examples of codes in the format \"{code}: {description}\". Please create new codes when needed:\n"
        # Shuffle codes to mitigate LLM recency bias
        code_desc_str = [f"{code}: {description}" for code, description in code_descriptions.items()]
        random.shuffle(code_desc_str)
        # Add each code as a new line
        prompt += "\n".join(code_desc_str) + "\n\n"

    prompt += "Below, I first give you examples of the output you should produce given an example input. After that, I give you the actual input to process.\n\n"

    # Add the few-shot examples in random order
    for _, row in few_shot_examples.sample(frac=1).iterrows():
        prompt += f"EXAMPLE INPUT:\n{row.text}\n\n"
        prompt += f"EXAMPLE OUTPUT:\n{row.coded_text}\n\n"

    prompt += f"ACTUAL INPUT:\n{text}"

    if insights is not None:
        prompt += f"\n\nMAIN INSIGHTS:\n{insights}"

    return prompt


def construct_deductive_prompt(text,
                               research_question,
                               codebook,
                               few_shot_examples):
    prompt = f"You are an expert qualitative researcher conducting a project with the research question: {research_question}. "
    prompt += """You are given a text to code deductively using a list of codes. Please carry out the following task:
- Respond by repeating the original text, but highlighting the coded statements by surrounding the statements with double asterisks, as if they were bolded text in a Markdown document.
- Include the associated code(s) immediately after the statement, separated by a semicolon and enclosed in <sup></sup> tags, as if they were superscript text in a Markdown document.
- Preserve exact formatting of the original text. Do not correct typos or remove unnecessary spaces.\n\n"""

    if few_shot_examples is None:
        prompt += "The following is an example of the correct output format, with fictional codes:\n\n"
        prompt += "I really enjoy walking in the park on weekends. **It helps me clear my mind**<sup>mental clarity</sup> and **feel more connected to nature**<sup>connection to nature</sup>. Sometimes, I take my dog with me, and we just wander around, **enjoying the fresh air**<sup>sensory enjoyment; relaxation</sup>.\n\n"

    prompt += "Use codes from the following list:\n" 
    # Shuffle codebook to mitigate LLM recency bias
    shuffled_codebook = codebook[:]
    random.shuffle(shuffled_codebook)
    # Add each code as a new line
    for item in shuffled_codebook:
        if len(item) == 1:
            # No description included, add only code
            prompt += f"{item[0]}\n"
        else:
            prompt += f"{item[0]}: {item[1]}\n"
    prompt += "\n"

    if few_shot_examples is not None:
        prompt += "Below, I first give you examples of the output you should produce given an example input. After that, I give you the actual input to process.\n\n"

        # Add the few-shot examples in random order
        for _, row in few_shot_examples.sample(frac=1).iterrows():
            prompt += f"EXAMPLE INPUT:\n{row.text}\n\n"
            prompt += f"EXAMPLE OUTPUT:\n{row.coded_text}\n\n"

    prompt += f"INPUT:\n{text}"
    
    return prompt


def construct_insight_prompt(text, research_question):
    prompt = f"You are an expert qualitative researcher who is given the following text to analyze:\n\n{text}\n\n"
    prompt += f"Output a single sentence summarising the most interesting insights in the text, specifically pertaining to the research question \"{research_question}\". "
    prompt += f"If there are no relevant insights, output \"The text contains no insights relevant to the research question.\""
    return prompt


def generate_code_description(code, examples, research_question, gpt_model, use_cache, counter_examples=[]):
    prompt = "Write a brief but nuanced one-sentence description for the given inductive code, based on a set of texts annotated with the code"

    if len(counter_examples) > 0:
        prompt += " and counter-examples where the code does not apply.\n"
    else:
        prompt += ".\n"

    prompt += " For example, for the code \"overcommunication\", you might generate the description: Captures instances where participants discuss feeling overwhelmed by excessive communication, such as constant emails, messages, or meetings\n"
    prompt += f" Write the description in the context of a qualitative research project with the research question: {research_question}.\n\n"

    prompt += f"CODE: {code}\n\n"

    prompt += "CODED TEXTS SEPARATED BY \"***\":\n"
    prompt += "\n***\n".join(examples)

    if len(counter_examples) > 0:
        prompt += "\n\nCOUNTER-EXAMPLES SEPARATED BY \"***\":\n"
        prompt += "\n***\n".join(examples)

    continuations = query_LLM(
        model=gpt_model,
        prompts=[prompt],
        max_tokens=200,
        use_cache=use_cache
    )
    
    return continuations[0]


def correct_coding_errors(texts, continuations, verbose=False):
    """
    Check for LLM errors and attempt to correct errors where the LLM has omitted non-coded parts of the original text.
    Outputs a list of LLM-coded texts corresponding to texts, with None in place of outputs that could not be corrected.
    """
    coded_texts = []
    n_reconstructed = 0

    for text, cont in zip(texts, continuations):
        if len(cont) == 0:
            # If the response is empty (possibly, due to model censorship), discard
            print(f"WARNING: Discarding empty LLM response for text \"{text}\"\n")
            coded_texts.append(None)
            continue

        # Check if there were any clear errors or hallucinations. In other words, does the LLM response (cont) match
        # the original text when we remove the highlight and code annotations?
        cont_text = re.sub(r"\*\*|<sup>(.*?)<\/sup>", "", cont)
        if cont_text == text:
            coded_texts.append(cont)
            continue

        # Sometimes, the LLM autocorrects typos even though we explicitly tell it not to.
        # Ignore the difference if the edit distance between the original and LLM-highlighted text is small enough.
        dist_threshold = 5
        edit_dist = distance(text, cont_text)
        if verbose:
            print(f"Warning: LLM output differs from the original text, with edit distance {edit_dist}")
            print(f"Original text: \"{text}\"")
            print(f"LLM output: \"{cont}\"")
        
        if edit_dist < dist_threshold:
            if verbose:
                print(f"Distance less than treshold {dist_threshold}, accept\n")
            coded_texts.append(cont)
            continue

        # Error detected, attempt to reconstruct by finding the annotations in the original text with fuzzy matching
        annotations = re.findall(r"\*\*(.*?)\*\*(<sup>.*?<\/sup>)", cont)
        reconstructed = text
        reconstruction_failed = False
        for highlight, codes in annotations:
            match_start, match_end, ratio = find_best_match(reconstructed, highlight)
            if ratio >= 90:
                # Add annotation to reconstruction if found a match at sufficient similarity ratio
                rec_annotation = "**" + reconstructed[match_start:match_end] + "**" + codes
                reconstructed = reconstructed[:match_start] + rec_annotation + reconstructed[match_end:]
            else:
                if verbose:
                    print(f"Could not find the LLM-annotated text \"{highlight}\" in the original text \"{text}\"")
                reconstruction_failed = True
                break

        if reconstruction_failed:
            if verbose:
                print(f"Text reconstruction failed, discard LLM response\n")
            coded_texts.append(None)
        else:
            if verbose:
                print(f"Text reconstruction successful\n")
            coded_texts.append(reconstructed)
            n_reconstructed += 1

    if n_reconstructed > 0 and verbose:
        print(f"Had to reconstruct {n_reconstructed} texts due to LLM errors")

    n_discarded = sum(t is None for t in coded_texts)
    if n_discarded > 0:
        print(f"WARNING: A total of {n_discarded} LLM outputs were discarded because of errors\n")

    return coded_texts


def find_best_match(original_text, target_substring):
    """
    Finds the substring in original_text that best matches target_substring using
    fuzzy matching, allowing for some length variance.

    Args:
        original_text (str): The text to search in.
        target_substring (str): The string to find a similar match for.

    Returns:
        tuple: (best_start, best_end, best_ratio) where best_start and best_end are
        the indices of the best matching substring, and best_ratio is the similarity score.
    """

    best_ratio = 0
    best_start = 0
    best_end = 0

    # We only search for matches within max_length_variance from len(target_substring)
    max_length_variance = int(len(target_substring) / 4)

    # Iterate over all possible window sizes
    for start in range(len(original_text)):
        end_min = start + len(target_substring) - max_length_variance
        end_max = min(start + len(target_substring) + max_length_variance, len(original_text) + 1)
        for end in range(end_min, end_max):
            window = original_text[start:end]
            similarity_ratio = fuzz.ratio(window, target_substring)
            
            # Update the best match if this window has a better match
            if similarity_ratio > best_ratio:
                best_ratio = similarity_ratio
                best_start = start
                best_end = end

    return best_start, best_end, best_ratio


def get_codes_and_highlights(coded_texts):
    code_highlights = defaultdict(list)
    for coded_text in coded_texts:
        for highlight, code in parse_codes(coded_text):
            code_highlights[code].append(highlight)
    return code_highlights


def parse_codes(coded_text):
    if coded_text is None:
        return []
    return [(highlight, code.strip()) 
            for highlight, codes in re.findall(r"\*\*(.*?)\*\*<sup>(.*?)<\/sup>", coded_text)
            for code in codes.split(";")]


def get_2d_code_embeddings(codes, embedding_context, embedding_model, verbose=False):
    # Add context to each set of codes before embedding
    codes_with_context = [code + embedding_context for code in codes]
    embeddings = embed(codes_with_context, model=embedding_model, verbose=verbose)

    # Reduce embedding dimensionality to 2
    embeddings_2d = reduce_embedding_dimensionality(embeddings=embeddings, num_dimensions=2, verbose=verbose)

    # Return results as a DataFrame
    df_em = pd.DataFrame([(code,) for code in codes], columns=["code"])
    df_em["code_2d_0"] = embeddings_2d[:,0]
    df_em["code_2d_1"] = embeddings_2d[:,1]
    return df_em


In [49]:
import os

os.environ["OPENAI_API_KEY"] = key.openAI_key

In [50]:
print("sheet_name:", sheet_name)

# Remove rows with NaN in the key columns as they cannot be used as dictionary keys
#> not sensitive to multiple identical keys: dict(zip(df['constant'], df['constant_comments']))
df = all_sheets_RR[sheet_name]

constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
print("mapping constant x comments:", constant_comments_mapping)
print(len(constant_comments_mapping))

new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
print("mapping new x comments:", new_comments_mapping)
print(len(new_comments_mapping))

deleted_comments_mapping = create_multivalue_dict(df, 'deleted', 'deleted_comments')
print("mapping deleted x comments:", deleted_comments_mapping)
print(len(deleted_comments_mapping))

sheet_name: SA
mapping constant x comments: {'safety': ['Prevents people from having to work under difficult or dangerous conditions to increase safety.', 'Enables deployment in difficult situations where human rescuers would put themselves in danger', nan, nan, nan, 'through reliability', 'No people are being put in danger'], 'remote control': ['Can be operated by experts via remote control or autonomous systems to perform operations from a safe distance.', nan, nan, nan, nan, 'Minimizes risks associated with autonomy'], 'speed and efficiency': ['Robots can often act faster than humans and increase the speed of emergency interventions', 'Improvement of speed and efficiency of rescue operations', nan], 'safe': ['People do not have to put themselves in dangerous situations'], 'reliable': ['Can be precisely controlled', nan, nan, nan, nan], 'no people necessary': [nan], 'stronger than humans': [nan, 'In many situations, e.g. when clearing rubble, a great advantage.', nan, nan], 'faster t

In [51]:
constant_new_comments_mapping = combine_dicts(constant_comments_mapping, new_comments_mapping)
print("mapping constant, new x comments:", constant_new_comments_mapping)
print(len(constant_new_comments_mapping))


mapping constant, new x comments: {'safety': ['Prevents people from having to work under difficult or dangerous conditions to increase safety.', 'Enables deployment in difficult situations where human rescuers would put themselves in danger', nan, nan, nan, 'through reliability', 'No people are being put in danger'], 'remote control': ['Can be operated by experts via remote control or autonomous systems to perform operations from a safe distance.', nan, nan, nan, nan, 'Minimizes risks associated with autonomy'], 'speed and efficiency': ['Robots can often act faster than humans and increase the speed of emergency interventions', 'Improvement of speed and efficiency of rescue operations', nan], 'safe': ['People do not have to put themselves in dangerous situations'], 'reliable': ['Can be precisely controlled', nan, nan, nan, nan], 'no people necessary': [nan], 'stronger than humans': [nan, 'In many situations, e.g. when clearing rubble, a great advantage.', nan, nan], 'faster than humans

In [52]:
init(API="OpenAI")

In [53]:
import numpy as np

# Sample dictionary with NaN values
dictionary = constant_new_comments_mapping

# Initialize a set to store unique single concepts
unique_concepts = set()

# Iterate through the dictionary
for key, values in dictionary.items():
    for value in values:
        # Check if the value is not NaN
        if isinstance(value, str):
            unique_concepts.add(f"{key}: {value}")
        else:
            unique_concepts.add(key)

# Convert the set to a list and take only the first 10 elements
single_concepts = unique_concepts # list(unique_concepts)[:10]

# Print the result
print(single_concepts)


{'less risk of injury', 'exclusion of human error', 'remote control', 'stronger than human: Specially equipped machines could help trapped victims quicker and more efficiently.', 'safety: No people are being put in danger', 'can save lives', 'enter dangerous areas', 'reliable: Can be precisely controlled', 'accessibility of special regions', 'by injuries', 'flexible: The robot could operate in dangerous areas and is therefore not limited.', 'deployment possibility: Also possible under dangerous conditions', 'reduce number of victims: Robots could make rescue operations more efficient and faster, potentially saving more victims.', 'precise work', 'rescue forces safety: Emergency services can avoid dangers if, for example, pictures of the location were taken by robots.', 'discover dangerous areas', 'error reduction', 'access to areas: Access to areas difficult or only reachable through considerable dangers for people', 'faster help: Since the risk to life is lower, rescue workers can sen

In [54]:
print(len(single_concepts))

225


In [55]:
single_concepts = list(unique_concepts)[:30]
print(len(single_concepts))

30


In [56]:
single_concepts

['less risk of injury',
 'exclusion of human error',
 'remote control',
 'stronger than human: Specially equipped machines could help trapped victims quicker and more efficiently.',
 'safety: No people are being put in danger',
 'can save lives',
 'enter dangerous areas',
 'reliable: Can be precisely controlled',
 'accessibility of special regions',
 'by injuries',
 'flexible: The robot could operate in dangerous areas and is therefore not limited.',
 'deployment possibility: Also possible under dangerous conditions',
 'reduce number of victims: Robots could make rescue operations more efficient and faster, potentially saving more victims.',
 'precise work',
 'rescue forces safety: Emergency services can avoid dangers if, for example, pictures of the location were taken by robots.',
 'discover dangerous areas',
 'error reduction',
 'access to areas: Access to areas difficult or only reachable through considerable dangers for people',
 'faster help: Since the risk to life is lower, resc

In [57]:
import nest_asyncio
nest_asyncio.apply()



# Sample inputs for function arguments

texts = single_concepts

research_question = "What are the key perceived benefits and risks regarding safety of the search and rescue robot?"

few_shot_examples = pd.DataFrame({
    "text": [
        "I feel valued and appreciated by my team.",
        "The training programs are really beneficial."
    ],
    "coded_text": [
        "**I feel valued and appreciated**<sup>employee recognition</sup> by my team.",
        "The **training programs**<sup>professional development</sup> are really beneficial."
    ]
})

gpt_model = "gpt-4o"  # or another preferred GPT model
use_cache = True
max_tokens = 150  # Specify maximum tokens for each prompt if necessary
verbose = True

# Now you can run:
coded_texts, code_descriptions = code_inductively_with_code_consistency(
    texts=texts,
    research_question=research_question,
    few_shot_examples=few_shot_examples,
    gpt_model=gpt_model,
    use_cache=use_cache,
    max_tokens=max_tokens,
    verbose=verbose
)


 |███-------------------------------------------------------------------------------------------------| 3.3% 

insight_prompt:
 You are an expert qualitative researcher who is given the following text to analyze:

less risk of injury

Output a single sentence summarising the most interesting insights in the text, specifically pertaining to the research question "What are the key perceived benefits and risks regarding safety of the search and rescue robot?". If there are no relevant insights, output "The text contains no insights relevant to the research question."
N: 1
prompt_batch for loop: ['You are an expert qualitative researcher who is given the following text to analyze:\n\nless risk of injury\n\nOutput a single sentence summarising the most interesting insights in the text, specifically pertaining to the research question "What are the key perceived benefits and risks regarding safety of the search and rescue robot?". If there are no relevant insights, output "The text contains 

In [58]:
len(coded_texts)

30

In [59]:
print(len(code_descriptions))
print(code_descriptions)

2
{'search and rescue robots': 'Captures discussions where participants describe search and rescue robots as specially equipped machines designed to enhance safety and efficiency in emergency situations by navigating hazardous environments and performing tasks that may be too dangerous for human responders.', 'key benefit': 'Captures instances where participants highlight the perceived advantage of search and rescue robots in significantly reducing the time and effort required to locate and assist trapped victims, thereby enhancing the overall efficiency of rescue operations.'}


{'robot endurance': "Captures discussions where participants highlight the robot's ability to operate for extended periods without fatigue, emphasizing its potential to enhance search and rescue missions by maintaining consistent performance over long durations.",
 'area of application': 'Captures discussions about specific environments or situations where the search and rescue robot is deemed particularly beneficial, such as hazardous locations that pose significant risks to human safety.',
 'reduction in human deployment': 'Captures instances where participants highlight the perceived safety benefit of minimizing human exposure to hazardous environments by deploying search and rescue robots, thereby reducing the need for human personnel to enter potentially dangerous situations.',
 'enhancing safety': 'Captures instances where participants highlight the benefit of minimizing human exposure to hazardous environments by utilizing search and rescue robots, thereby enhancing overall safety during operations.',
 'rapid deployment': "Captures instances where participants highlight the advantage of the search and rescue robot's ability to be quickly deployed to disaster sites, significantly reducing response time compared to human teams.",
 'simplification of difficult situations': 'Captures instances where participants highlight how search and rescue robots can streamline complex rescue operations by providing automated solutions that enhance accessibility and intervention in challenging environments.',
 'key benefit': 'Captures instances where participants highlight the primary advantage of the search and rescue robot as its ability to save lives by efficiently locating and assisting individuals in emergency situations.',
 'robot focus': 'Captures instances where participants emphasize the importance of the search and rescue robot maintaining a singular focus on its task to ensure efficiency and safety, avoiding distractions that could compromise its performance in critical situations.',
 'replaceability': 'Captures discussions where participants highlight the perceived benefit of robots in search and rescue operations being easily replaceable, thus reducing concerns about their safety compared to human rescuers.',
 'robot capability': "Captures discussions where participants highlight the robot's ability to perform complex tasks with precision and agility, emphasizing its potential to enhance safety in search and rescue operations by navigating challenging environments effectively."}

In [61]:
coded_texts_deductively = code_deductively(texts,
                     research_question=research_question,
                     codebook=code_descriptions,
                     gpt_model=gpt_model,
                     few_shot_examples=None,
                     use_cache=True,
                     verbose=False)

N: 30
prompt_batch for loop: ['You are an expert qualitative researcher conducting a project with the research question: What are the key perceived benefits and risks regarding safety of the search and rescue robot?. You are given a text to code deductively using a list of codes. Please carry out the following task:\n- Respond by repeating the original text, but highlighting the coded statements by surrounding the statements with double asterisks, as if they were bolded text in a Markdown document.\n- Include the associated code(s) immediately after the statement, separated by a semicolon and enclosed in <sup></sup> tags, as if they were superscript text in a Markdown document.\n- Preserve exact formatting of the original text. Do not correct typos or remove unnecessary spaces.\n\nThe following is an example of the correct output format, with fictional codes:\n\nI really enjoy walking in the park on weekends. **It helps me clear my mind**<sup>mental clarity</sup> and **feel more connec

In [62]:
coded_texts_deductively

['less **risk of injury**<sup>key benefit</sup>',
 '**exclusion of human error**<sup>key benefit</sup>',
 'remote control',
 'stronger than human: **Specially equipped machines could help trapped victims quicker and more efficiently.**<sup>key benefit; search and rescue robots</sup>',
 'safety: **No people are being put in danger**<sup>key benefit</sup>',
 'can **save lives**<sup>key benefit</sup>',
 'enter **dangerous areas**<sup>search and rescue robots; key benefit</sup>',
 'reliable: **Can be precisely controlled**<sup>key benefit</sup>',
 'accessibility of special regions',
 'by injuries',
 'flexible: **The robot could operate in dangerous areas**<sup>search and rescue robots; key benefit</sup> and is therefore not limited.',
 'deployment possibility: **Also possible under dangerous conditions**<sup>key benefit; search and rescue robots</sup>',
 'reduce number of victims: **Robots could make rescue operations more efficient and faster, potentially saving more victims.**<sup>key be

In [63]:
#@title Show distribution of LLM-generated deductive codes

# Parse all codes and highlights in LLM output
code_highlights_ded = get_codes_and_highlights(coded_texts_deductively)


In [64]:
code_highlights_ded

defaultdict(list,
            {'key benefit': ['risk of injury',
              'exclusion of human error',
              'Specially equipped machines could help trapped victims quicker and more efficiently.',
              'No people are being put in danger',
              'save lives',
              'dangerous areas',
              'Can be precisely controlled',
              'The robot could operate in dangerous areas',
              'Also possible under dangerous conditions',
              'Robots could make rescue operations more efficient and faster, potentially saving more victims.',
              'Emergency services can avoid dangers if, for example, pictures of the location were taken by robots.',
              'dangerous areas',
              'Access to areas difficult or only reachable through considerable dangers for people',
              'Since the risk to life is lower, rescue workers can send a robot before a complete analysis of the problem area is available.',
        

In [65]:
import plotly.express as px


# @title
def plot_generated_codes(code_highlights, title):
    code_counts = [(code, len(highlights)) for code, highlights in code_highlights.items()]
    df_codes = pd.DataFrame(code_counts, columns=['Code', 'Count'])
    df_codes = df_codes.sort_values(by='Count', ascending=False).reset_index(drop=True)

    # Create a vertical bar plot using Plotly with angled x-axis labels
    fig = px.bar(df_codes, x='Code', y='Count', title=title)

    # Update layout to angle x-axis labels at 45 degrees
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

# Parse all codes and highlights in LLM output
plot_generated_codes(code_highlights_ded, 'LLM-generated inductive codes')

In [ ]:
ERROR

In [ ]:
system_template = """
<Context> You are a researcher tasked with summarizing a list of words into generic/superordinate categories. Based on these categories, create a dictionary that assigns the respective subordinate terms (keys from the provided "overallList") to the generic terms. Laypersons were informed about the potential risks and benefits of rigid and soft {robots} through scenario texts. They then listed their perceived risks and benefits of rigid and soft robots in the "overallList" wordlist. The overarching topic of the list is {topicCategory}, whereby the topic involved {topicCategoryDetails}.</Context>

<Data Structure> The list "overallList" is a dictionary where the keys are written arguments, and the corresponding values are one or more comments related to those arguments. The value [nan] indicates that no specific comment was provided for the respective entry. If there are multiple comments or missing entries ([nan]), it signifies that the respective argument was mentioned multiple times, emphasizing its importance. </Data Structure>

<Task> Your task is to create two outputs:
1. A list called "listGeneric" that contains the generic/superordinate categories. You may use no more than six different categories.
2. A dictionary called "dictionary" that contains: Keys (the generic/superordinate categories) and values (the corresponding words - keys - from the "overallList" that have been summarized under each category).
The dictionary must contain all corresponding words (keys) from the "overallList". If it is not possible to assign a specific word, please place it in a category called "rest category".</Task>
"""


user_template = """
List "overallList": 
{overallList}
"""

Prompt to apply coding guidelines:

In [ ]:
# aaa

In [ ]:
ERROR



###############################################

# aaaaaaaaaaaaaaaaaaaa

### for task to get superordinate categories within single categories

In [ ]:
system_template = """
<Context>
You are a researcher tasked with summarizing a list of words into generic/superordinate categories. Based on these categories, create a dictionary that assigns the respective subordinate terms (keys from the provided "overallList") to the generic terms. Laypersons were informed about the potential risks and benefits of rigid and soft {robots} through scenario texts. They then listed their perceived risks and benefits of rigid and soft robots in the "overallList" wordlist. The overarching topic of the list is {topicCategory}.
</Context>

<Data Structure>
The list "overallList" is a dictionary where the keys are written arguments, and the corresponding values are one or more comments related to those arguments. The value [nan] indicates that no specific comment was provided for the respective entry. If there are multiple comments or missing entries ([nan]), it signifies that the respective argument was mentioned multiple times, emphasizing its importance.
</Data Structure>

<Task>
Your task is to create two outputs:
1. A list called "listGeneric" that contains the generic/superordinate categories. You may use no more than six different categories.
2. A dictionary called "dictionary" that contains:
   - Keys: The generic/superordinate categories.
   - Values: The corresponding words (keys) from the "overallList" that have been summarized under each category.
The dictionary must contain all corresponding words (keys) from the "overallList". If it is not possible to assign a specific word, please place it in a category called "rest category".
</Task>
"""

# !!!
system_template = """
<Context> You are a researcher tasked with summarizing a list of words into generic/superordinate categories. Based on these categories, create a dictionary that assigns the respective subordinate terms (keys from the provided "overallList") to the generic terms. Laypersons were informed about the potential risks and benefits of rigid and soft {robots} through scenario texts. They then listed their perceived risks and benefits of rigid and soft robots in the "overallList" wordlist. The overarching topic of the list is {topicCategory}, whereby the topic involved {topicCategoryDetails}.</Context>

<Data Structure> The list "overallList" is a dictionary where the keys are written arguments, and the corresponding values are one or more comments related to those arguments. The value [nan] indicates that no specific comment was provided for the respective entry. If there are multiple comments or missing entries ([nan]), it signifies that the respective argument was mentioned multiple times, emphasizing its importance. </Data Structure>

<Task> Your task is to create two outputs:
1. A list called "listGeneric" that contains the generic/superordinate categories. You may use no more than six different categories.
2. A dictionary called "dictionary" that contains: Keys (the generic/superordinate categories) and values (the corresponding words - keys - from the "overallList" that have been summarized under each category).
The dictionary must contain all corresponding words (keys) from the "overallList". If it is not possible to assign a specific word, please place it in a category called "rest category".</Task>
"""




user_template = """
List "overallList": 
{overallList}
"""

# rescue robots and socially assistive robots
prompt_template_SC = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

constant_new_comments_mapping = combine_dicts(constant_comments_mapping, new_comments_mapping)

result = prompt_template_SC.invoke({"robots": "rescue robots and socially assistive robots", "topicCategory": abbreviations_dict[sheet_name], "overallList": constant_new_comments_mapping})
print(result)

print("result:", result)
print("result.to_messages():", result.to_messages())

## Provide schemas for structured outputs

### for task to get main findings, differences and summary

In [ ]:
json_schema = {
    "title": "Outputs",
    "description": "Bullet lists detailing the similarities and differences between the rigid and soft lists and a summary paragraph.",
    "type": "object",
    "properties": {
        "mainFindings": {
            "type": "string",
            "description": "Bullet lists highlighting the main findings of the provided rigid and soft lists",
        },
        "differences": {
            "type": "string",
            "description": "Bullet lists detailing the differences between the rigid and soft lists",
        },
          "summary": {
            "type": "string",
            "description": "Summary paragraph that provides a summary of the main findings and the found differences",
        },
    },
    "required": ["similarities", "differences", "summary"],
}

### for task to get superordinate categories within single categories

In [ ]:
json_schema_notUsed = {
    "title": "Outputs",
    "description": "List that contains the generic / superordinate categories and a dictionary, which assigns the respective subordinate terms to the generic terms.",
    "type": "object",
    "properties": {
        "listGeneric": {
            "type": "string",
            "description": "List that contains the generic / superordinate categories",
        },
        "dictionary": {
            "type": "string",
            "description": "Dictionary that contains the keys, the generic / superordinate categories and the corresponding words that have been summarised under the respective category",
        },
    },
    "required": ["listGeneric", "dictionary"],
}

In [ ]:
json_schema_SC = {
    "title": "Outputs",
    "description": "List that contains the generic/superordinate categories and a dictionary that assigns the respective subordinate terms to the generic terms.",
    "type": "object",
    "properties": {
        "listGeneric": {
            "type": "array",
            "description": "List that contains the generic/superordinate categories.",
            "items": {
                "type": "string"
            }
        },
        "dictionary": {
            "type": "object",
            "description": "Dictionary that contains the generic/superordinate categories as keys and the corresponding words from the 'overallList' as values.",
            "additionalProperties": {
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        }
    },
    "required": ["listGeneric", "dictionary"]
}

## Define basic API call

### for task to get main findings, differences and summary

In [ ]:
def basic_API_call(
    prompt,
    robots,
    topicCategory,
    openai_api_key,
    dictonaryRigid,
    dictonarySoft,
    json_schema,
    model_name="gpt-4o",
    max_tokens=1000,
):

    # prompt = PromptTemplate(template=template)
    seed = 123

    model = ChatOpenAI(model=model_name, openai_api_key=openai_api_key, max_tokens=max_tokens, model_kwargs={"seed": seed}, temperature=0.0)
       
    structured_llm = model.with_structured_output(json_schema, include_raw=True)
    chain = prompt | structured_llm

    with get_openai_callback() as cb:
        response = chain.invoke(
            {"robots": robots, "topicCategory": topicCategory, "rigid": dictonaryRigid, "soft": dictonarySoft}
        )
        print(cb)
    
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
        
    return response

### for task to get superordinate categories within single categories

In [ ]:
def basic_API_call_SC(
    prompt,
    robots,
    topicCategory,
    openai_api_key,
    dictonaryCombined,
    json_schema,
    model_name="gpt-4o",
    max_tokens=1000,
):

    # prompt = PromptTemplate(template=template)
    seed = 123

    model = ChatOpenAI(model=model_name, openai_api_key=openai_api_key, max_tokens=max_tokens, model_kwargs={"seed": seed}, temperature=0.0)
       
    structured_llm = model.with_structured_output(json_schema, include_raw=True)
    chain = prompt | structured_llm

    with get_openai_callback() as cb:
        response = chain.invoke(
            {"robots": robots, "topicCategory": topicCategory, "overallList": dictonaryCombined}
        )
        print(cb)
    
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
        
    return response

# Run ChatGPT

## Example (overall)

only for main findings, difference, summary

> Remark: The argument structures between the two types of robots differ significantly. Therefore, the robots are qualitatively summarized separately to ensure a clear and accurate comparison.

In [ ]:
print(f"sheet_name: {sheet_name}")
print(f"abbreviations_dict[sheet_name]: {abbreviations_dict[sheet_name]}")


result = basic_API_call(prompt=prompt_template,
    robots="rescue robots and socially assistive robots",
    topicCategory=abbreviations_dict[sheet_name],
    openai_api_key=key.openai_api_key,
    dictonaryRigid=constant_comments_mapping, # overall
    dictonarySoft=new_comments_mapping,
    json_schema=json_schema,
    model_name="gpt-4o",
    max_tokens=1000,
)

In [ ]:
# Extract the 'parsed' section from the JSON data
parsed_section = result.get('parsed', {})
#print(parsed_section)
# Extract the translations
mainFindings = parsed_section.get('mainFindings')
differences = parsed_section.get('differences')
summary = parsed_section.get('summary')

print(f"result (raw): {result}")
print(f"mainFindings: {mainFindings}")
print(f"differences: {differences}")
print(f"summary: {summary}")

## Separately for robots (rescue robot and socially assistive robot)

### for rescue robots (main findings, difference, summary)

In [ ]:
categories = []
mainFindings = []
differences = []
summary = []
rawResults = []


for category in abbreviations_dict.keys():
    print(f"category: {category}")
    
    # do not process the rest categories
    if category not in ['RCPP', 'RCPN', 'RCA', 'RCN']:
        df = all_sheets_RR[category]
        constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
        # print("mapping constant x comments:", constant_comments_mapping)
        # print(len(constant_comments_mapping))
        
        new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
        # print("mapping new x comments:", new_comments_mapping)
        # print(len(new_comments_mapping))
    
        result = basic_API_call(prompt=prompt_template,
            robots="rescue robots",
            topicCategory=abbreviations_dict[category],
            openai_api_key=key.openai_api_key,
            dictonaryRigid=constant_comments_mapping,
            dictonarySoft=new_comments_mapping,
            json_schema=json_schema,
            model_name="gpt-4o",
            max_tokens=1600, # increase limit
        )
        
        # append raw results
        categories.append(category)
        rawResults.append(result)
        
        # append parsed results
        parsed_section = result.get('parsed', {})
        mainFindings.append(parsed_section.get('mainFindings'))
        differences.append(parsed_section.get('differences'))
        summary.append(parsed_section.get('summary'))
        #print("length of mainFindings:", len(parsed_section.get('mainFindings')))
        #print("length of differences:", len(parsed_section.get('differences')))
        #print("length of summary:", len(parsed_section.get('summary')))
        
# save file
df_RR = pd.DataFrame({
    'Category': categories,
    'mainFindings': mainFindings,
    'differences': differences,
    'summary': summary,
    'rawResults' : rawResults
})

# Path to your Excel file
file_path = directory + "/output/" + "rescue robot_ChatGPT" + ".xlsx"
# save the dataframe to an Excel file
df_RR.to_excel(file_path, index=False)

### for rescue robots (get superordinate categories within single categories: listGeneric, dictionary)

In [ ]:
categories = []
mainFindings = []
differences = []
summary = []
rawResults = []


for category in abbreviations_dict.keys():
    print(f"category: {category}")
    
    # do not process the rest categories
    # not in ['RCPP', 'RCPN', 'RCA', 'RCN']:
    if category in ['MT']:
        df = all_sheets_RR[category]
        constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
        # print("mapping constant x comments:", constant_comments_mapping)
        # print(len(constant_comments_mapping))
        
        new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
        # print("mapping new x comments:", new_comments_mapping)
        # print(len(new_comments_mapping))
        constant_new_comments_mapping = combine_dicts(constant_comments_mapping, new_comments_mapping)


        result = basic_API_call_SC(prompt=prompt_template_SC,
            robots="rescue robots",
            topicCategory=abbreviations_dict[category],
            openai_api_key=key.openai_api_key,
            dictonaryCombined=constant_new_comments_mapping,
            json_schema=json_schema_SC,
            model_name="gpt-4o",
            max_tokens=2000, # increase limit
        )
        
        # append raw results
        categories.append(category)
        rawResults.append(result)
        
        # append parsed results
        #parsed_section = result.get('parsed', {})
        #mainFindings.append(parsed_section.get('mainFindings'))
        #differences.append(parsed_section.get('differences'))
        #summary.append(parsed_section.get('summary'))
        #print("length of mainFindings:", len(parsed_section.get('mainFindings')))
        #print("length of differences:", len(parsed_section.get('differences')))
        #print("length of summary:", len(parsed_section.get('summary')))
        


In [ ]:
parsed_section = result.get('parsed', {})
print(parsed_section)

In [ ]:
# save file
df_RR = pd.DataFrame({
    'Category': categories,
    'mainFindings': mainFindings,
    'differences': differences,
    'summary': summary,
    'rawResults' : rawResults
})

# Path to your Excel file
file_path = directory + "/output/" + "rescue robot_ChatGPT" + ".xlsx"
# save the dataframe to an Excel file
df_RR.to_excel(file_path, index=False)

### for socially assistive robots (main findings, difference, summary)

In [ ]:
categories = []
mainFindings = []
differences = []
summary = []
rawResults = []


for category in abbreviations_dict.keys():
    print(f"category: {category}")
    
    # do not process the rest categories
    if category not in ['RCPP', 'RCPN', 'RCA', 'RCN']:
        df = all_sheets_SAR[category]
        constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
        # print("mapping constant x comments:", constant_comments_mapping)
        # print(len(constant_comments_mapping))
        
        new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
        # print("mapping new x comments:", new_comments_mapping)
        # print(len(new_comments_mapping))
    
        result = basic_API_call(prompt=prompt_template,
            robots="socially assistive robots",
            topicCategory=abbreviations_dict[category],
            openai_api_key=key.openai_api_key,
            dictonaryRigid=constant_comments_mapping,
            dictonarySoft=new_comments_mapping,
            json_schema=json_schema,
            model_name="gpt-4o",
            max_tokens=1000,
        )
        
        # append raw results
        categories.append(category)
        rawResults.append(result)
        
        # append parsed results
        parsed_section = result.get('parsed', {})
        mainFindings.append(parsed_section.get('mainFindings'))
        differences.append(parsed_section.get('differences'))
        summary.append(parsed_section.get('summary'))
        
# save file
df_SAR = pd.DataFrame({
    'Category': categories,
    'mainFindings': mainFindings,
    'differences': differences,
    'summary': summary,
    'rawResults' : rawResults
})

# Path to your Excel file
file_path = directory + "/output/" + "socially assistive robot_ChatGPT" + ".xlsx"
# save the dataframe to an Excel file
df_SAR.to_excel(file_path, index=False)